# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.14 用AI开发AI（下）：全自动代码审查流程与复杂任务管理方法

- 软件开发过程利用大语言模型提高开发效率的不同阶段

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/702fe4b67f1781de14a5e217a2d923b.png" alt="702fe4b67f1781de14a5e217a2d923b" style="zoom:33%;" />

In [1]:
import os
import openai
import glob
import shutil
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser

import sys
sys.path.insert(0, '.\\functions\\untested functions')
sys.path.insert(0, '.\\functions\\tested functions')
from gptLearning import *

### 1.基于大语言模型的自动代码审查流程

- 大语言模型debug能力验证

&emsp;&emsp;一个非常有趣的现象是，很多时候大语言模型和人类一样，“做作业”（根据提示生成答案）的时候会犯迷糊，但如果“自己再检查一遍”，就能够很好的检查出此前回答的答案中存在的问题，并且，由于大语言模型并不存在长期记忆，因此哪怕是用大语言模型审查自己输出的内容，其实也相当于自己去检查别人的作业，查错效率很高。例如围绕上一小节生成的错误代码get_first_email（注意是保存在wrong code文件夹内的错误的get_first_email函数，并不是在untested文件夹中的函数），我们让Chat模型自行再次进行检查，测试其能否查出错误：

In [8]:
with open('./functions/wrong code/get_first_email/get_first_email_module.py' , encoding='utf-8') as f:
    content = f.read()

In [9]:
content

'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n    \n    # 获取第一封邮件的 Id\n    msg_id = messages[-1]["id"]\n\n    msg = service.users().messages().get(userId=\'me\', id=msg_id).execute()\n        \n    # 获取邮件内容\n    payload = msg[\'payload\']\n    headers = payload[\'headers\']\n    \n    for header in headers:\n        name = header[\'name\']\n        if name == "Date":\n            message_date = header[\'value\']\n            # 解析获得的时间字符串\n            message_timestamp = parsedate(message_date)\n

In [22]:
messages = [{"role": "user",  "content": "现有函数如下：%s，该函数是否有和问题，应该如何修改？" % content}]

In [23]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=messages
)

In [24]:
response.choices[0].message['content']

'该函数中存在的问题和修改建议如下：\n\n1. 函数声明后的一部分，存在多余的转义字符`\\`，应当去除。\n\n2. 函数参数默认值为 \'me\' ，但在函数内部获取邮件的时候直接使用了 \'me\' ，没有使用到传入的参数 `userId`。\n\n3. `Credentials.from_authorized_user_file()` 的文件路径为 \'token.json\' ，如果这个 token 文件不在当前目录或者没有该文件，函数运行会报错。\n\n4. 这个函数假设了 `service.users().messages().list(userId=userId).execute()` 返回的结果一定有 \'messages\' 这个字段，如果没有，可能会出现异常。\n\n5. 虽然在头部有说明返回包含时间和内容的json字符串，但收件时间和邮件内容在同一层循环中获取，如果邮件的 header 中没有 "Date"或"Subject"会因为没有定义 message_date 或者 subject 而引发错误。\n\n所以，该函数可以修改为：\n```python\ndef get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\n    注意，当查询我的邮箱时，userId需要输入\'me\';\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据，这里假设 "token.json" 文件存在\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).

这里我们手动提取修改之后的代码部分，然后进行运行，测试新函数能否顺利运行：

In [25]:
s = 'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\n    注意，当查询我的邮箱时，userId需要输入\'me\';\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据，这里假设 "token.json" 文件存在\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n\n    if not messages:\n        return None\n\n    # 获取第一封邮件的 Id\n    msg_id = messages[-1]["id"]\n\n    msg = service.users().messages().get(userId=userId, id=msg_id).execute()\n\n    # 获取邮件内容\n    payload = msg[\'payload\']\n    headers = payload[\'headers\']\n\n    message_date = None\n    subject = None\n    \n    for header in headers:\n        name = header[\'name\']\n        if name == "Date":\n            message_date = header[\'value\']\n            # 解析获得的时间字符串\n\n        if name == "Subject":\n            subject = header[\'value\']\n\n    # 将邮件数据以 JSON 格式返回\n    result = {\n        "date": message_date,\n        "content": subject\n    }\n\n    return json.dumps(result)'

In [26]:
s

'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\n    注意，当查询我的邮箱时，userId需要输入\'me\';\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据，这里假设 "token.json" 文件存在\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n\n    if not messages:\n        return None\n\n    # 获取第一封邮件的 Id\n    msg_id = messages[-1]["id"]\n\n    msg = service.users().messages().get(userId=userId, id=msg_id).execute()\n\n    # 获取邮件内容\n    payload = msg[\'payload\']\n    headers = payload[\'headers\']\n\n    message_date = None\n    subject = None\n    \n    for header in headers:\n        name = header[\'name\']\n        if name == "Date":\n            message_date = header[\'

In [27]:
exec(s)

In [28]:
get_first_email('me')

'{"date": "Wed, 19 Jul 2023 02:22:43 -0700", "content": "Google Payments\\uff1aGeorgia Kinghorne\\u7684\\u7535\\u5b50\\u90ae\\u4ef6\\u5730\\u5740\\u5df2\\u66f4\\u65b0"}'

能够发现，Chat模型很好的纠正了此前get_first_email函数中存在的错误，并且debug过程分析的非常严谨。而且非常有意思的是，尽管审查加过并没有提及parsedate未被定义之事，但修改之后的代码确实很好的修正了这一点，并且修改之后的代码也可以顺利运行。这也侧面说明对于具备编程能力的大语言模型来说是具备代码纠错能力的。

> 而这其实也是对话模型和Code模型统一带来的好处，即我们可以通过对话的方式调用Chat模型的编程能力对程序进行审查。

#### 1.1 审查模型构建思路

&emsp;&emsp;尽管上述示例中，我们只提供了函数代码信息，模型即可顺利找出代码问题并进行了有效修改，但实际上，由于此时用于审查的大模型（以下简称审查模型）并不知道该函数创建的前后信息，有时修改之后的模型可能也无法达到要求。例如审查模型并不知道我们是使用Gmail API完成该功能，并且将授权文件本地保存为token.json文件，同时我们需要函数的输入和输出都是字符串格式等信息，因此审查模型在修改时也很有可能修改得到功能正常但不满足这些背景要求的函数。此外由于在进行函数创建时，为了积累成功函数的提示示例，我们不仅需要功能正常的函数代码，同时也需要各阶段的提示内容，因此对于一个报错函数，仅仅修改代码可能并不足够。基于此，我们不仅需要让审查模型审查代码，同时也需要让其审查整个提示流程，并在必要的时候对提示词进行修改。以此才能真正达到审查之目的，最终修改的结果才能直接使用。

&emsp;&emsp;尽管我们判断大语言模型应该是具备审查全流程的能力的，但具体要如何操作才能让一个大语言模型审查一整个提示过程和最终输出的代码呢？这里需要注意，如果我们知识简单的将提示流程及代码进行输入然后让审查模型进行修改，往往并不能达到目标，此时审查模型会因为不理解审查的目标而进行错误的修改，并且由于我们没有并且不太可能积累错误提示的修改示例，因此也无法通过Few-shot方式引导模型顺利完成审查。此外，在操作层面也需要注意，这里我们如果采用此前的流程、简单的直接用字符串形式读取包含代码的json对象，则会因为转化规则而不断围绕特殊符号创建多个转义符，而最终导致无法阅读，例如下图就是一个存在过多转义符而无法进一步进行转化和读取的示例<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/7e50f00773104d56f66b995a8d99cdb.png" alt="7e50f00773104d56f66b995a8d99cdb" style="zoom:33%;" />

&emsp;&emsp;那么到底应该如何设计这个审查流程，才能让审查模型顺利的完成审查任务呢？

&emsp;&emsp;首先，由于审查任务本身的特殊性，我们必须要让模型理解什么是多阶段提示、以及为什么要进行类似形式的多阶段提示，及我们需要提前通过system messages为模型输入一些背景信息。此外，由于我们不能直接用大语言模型读取转化为字符串的json对象，因此需要找到一种能够同时表示json、python代码以及字符串类型对象——也就是我们此前曾使用过的markdown类型对象，也就是说，接下来我们将以字符串形式表示markdown类型对象作为审查模型输出和输出的对象类型。

- 外部信息和外部挂载文档

&emsp;&emsp;当然，markdown类型对象的创建和使用并不复杂，相关使用方法在Ch.7中曾经介绍过，比较复杂的是要如何设计输入给审查模型“背景信息”，才能更好的让审查模型完成审查任务。需要注意的是，在非常多的大语言模型应用实践场景中，我们都需要为大语言模型输入一些额外的信息才能更好的引导其完成任务，有时我们输入额外信息是为了让大语言模型获取一些非原生知识库的信息（例如围绕本地知识库的问答系统），而另外一些时候则是为了强调当前任务需要用到众多原生知识库中的某些知识。而无论输入的信息是何性质，我们都可以称这些“背景信息”为外部挂载信息，而如果是以文档形式进行输入，这些文档则可称为模型外部挂载文档。很明显，我们现在就需要创建一个审查模型的外部挂载文档，用于为审查模型输入必要的项目信息从而辅助其完成审查。

&emsp;&emsp;需要注意的是，在不同的项目中，外部挂载文档的创建和输入形式都各不相同。考虑到每个大语言模型都有最大上下文输入限制，因此当信息过多（超过最大限制）时，如何输入这些信息本身就成了一项具备一定技术难度的事情。关于如何突破最大上下文限制，我们会在后续的课程中进行详细探讨，本节关于审查模型的外部挂载文档，我们尽量将其长度控制在4k内，方便审查模型一次性进行读取。

&emsp;&emsp;关于如何创建这个审查模型的外部挂载文档，首先我们能想到的最简单的方法就是将此前课程的全部相关内容导出为markdown然后作为外部挂载文档，但如此一来文档本身的长度肯定超过4k（甚至肯定超过16k）。因此我们需要考虑采用一种更加言简意赅的创造这个审查模型的外部挂载文档。

&emsp;&emsp;这里我们就需要介绍下大语言模型外部挂载文档的基本设计思路，考虑到大语言模型本身的海量训练样本，我们这里创建的外部挂载文档无论内容多少，都无法动摇其原始的知识库。因此如果我们是手动创建这个外部文档，那么这个外部文档的核心作用应该是引导模型重视某部分信息、同时补充模型本身不具备的信息。换个更加通俗的话来说，外部文档的创建更像是为了给模型说明规则，同时给与模型一些“启发”，而不是从头开始“教会模型如何做事”。接下来我们就以这样的基本原则来设计一个能够高效引导模型进行审查的外部文档。

- 检测模型本身具备的知识

&emsp;&emsp;如何引导和启发一个大语言模型去理解一些陌生的事物？就像老师教学一样，有两个简单高效的方法，其一是是用其已知的知识去类比未知的知识，其二则是将陌生事物视为某些一般概念的特例。例如，当前的外部挂载文档非常重要的一个作用就是要让大语言模型理解现在要审查的多段提示，即要引导大语言模型理解什么是多段提示，但如果从头开始进行相关概念介绍（例如从头介绍两个阶段的提示分别是什么、什么是LtM、什么是问题拆解等）会非常的繁琐且效率很低，因此这里我们可以将多段提示比喻成推理链，借助大语言模型本身对推理链的理解来理解什么是多段提示。当然，这里我们需要简单测试下大语言模型本身是否知道什么是推理链：

In [29]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[{"role": "user",  "content": "请问什么是推理链？"}]
)

In [30]:
response.choices[0].message['content']

'推理链是一种逻辑思考或证据推理的方式，通过一个个推论步骤，从已知的事实或前提，推导出未知的结论。每一个推论步骤都在上一个推论的基础上进行，形成一个连续的推理过程，如同链条一般，因此被称为推理链。在很多领域，如科学研究、法律审判、人工智能等，都会使用到推理链的方式进行思考和决策。'

而在外部挂载文档中，我们就可以简单将多段提示看成是一种推理链，并且是一种非常特殊的推理链，其特殊之处在于这个推理链专门用于创建一系列Gmail API相关函数。这里我们进一步验证如果将多段提示视作推理链的话，大语言模型能否直接理解：

In [84]:
function_name = 'get_latest_email'

In [85]:
with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

In [95]:
chain_of_prompt = "以下是个一个成功运行的推理链条，推理分为三个阶段，各阶段推理内容如下：%s。" % msg

In [96]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[{"role": "user",  "content": "以下是一个推理链，用于创建一个满足某个功能的函数，推理链如下：%s。请帮我介绍下这个推理链" % chain_of_prompt}]
)

In [97]:
response.choices[0].message['content']

'这个推理链是用户要求助手帮助其编写一个Python函数，用于查看其Gmail邮箱中的最后一封邮件信息。\n\n推理链分为三个阶段：\n\n第一个阶段，是问题定义阶段，用户表达了他的需求，希望能查看其Gmail邮箱里最后一封邮件的内容。助手从这个需求中分析出可以作为函数参数的实体，即需要查看的邮箱所有者。\n\n第二个阶段，是问题转化阶段，用户把前一个阶段的需求转化为一个明确的函数的编写需求。需要编写的函数是用来查看一个特定的Gmail邮箱中最后一封邮件的内容，函数的调用参数是用户的邮箱ID，返回结果是一个包含最后一封邮件内容的JSON格式的对象。且要求在函数中添加详细的中文函数说明文档。\n\n第三个阶段，助手根据用户给出的具体需求，编写出了一个满足需求的Python函数。函数名为get_latest_email，输入参数为userId，上传一个Gmail邮箱ID，返回的是一个包含最后一封邮件所有信息的JSON格式对象。在函数中，首先从本地文件中加载凭据，然后创建Gmail API客户端，接着列出用户的一封最新邮件，最后返回该邮件的详细信息。'

能够发现，模型能够理解以prompt.json类型对象表示的推理链。

#### 1.2 审查模型外部挂载文档创建过程

&emsp;&emsp;在以类比推理链为基本思路，并且验证了大语言模型本身能够理解推理链及prompt.json之后，接下来我们只需要额外补充一些项目背景信息、一个成功的推理链样式、以及审查模型本身的身份设定并说明审查模型的工作职责，即可构成一个完整审查模型的外部挂载文档。正如此前所说，文档需要以markdown形式进行编写，文档名为《推理链修改》，文档如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d130318b6f587922701b73d0d8c816a.png" alt="d130318b6f587922701b73d0d8c816a" style="zoom:33%;" />

In [6]:
with open('推理链修改.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
print(md_content)

1.什么是智能邮件项目？智能邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮件操作，如查阅邮件、收发邮件等。项目采用的邮箱API是Gmail API，并且已经获得了授权，授权文件为本地的token.json文件。

2.什么是推理链？在智能邮件项目中，推理链特指将用户的需求逐步推导为python函数的过程，也就是说，每个推理链的输入都是用户需求，而推理的结果则是一个可以满足用户需求的函数。每个推理链都有三个推理环节，且都以json格式表示，例如，以下是一个正确的推理链A：

```json
{"stage1_CD": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bGmail\u90ae\u7bb1\u91cc\u6700\u540e\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002"}, {"role": "assistant", "content": "\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}], "stage1_CM": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bGmail\u90ae\u7bb1\u91cc\u6700\u540e\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}, {"role": "assistant", "content": "\u8bf7\u5e2e\u6211\u7f16\u5199\u4e00\u4e2apython\u51fd\u6570\uff0c\u7528\u4e8

&emsp;&emsp;这里我们可以简单总结外部挂载文档的5方面信息：1.说明当前项目背景（包括编程要求等）；2.说明审查对象，包括审查对象的性质和形式等；3.说明审查的目标；4.进行身份设置；5.说明审查之后的输出结果。当然从我能当长度来看，很明显，这个外部挂载文档的长度肯定不到4k，是可以一次性以system messages输入到Chat模型中的：

In [3]:
import tiktoken

In [4]:
encoding = tiktoken.encoding_for_model("gpt-4-0613")

In [5]:
len(encoding.encode(md_content))

2670

而接下来，我们即可借助该外部挂载文档来引导审查模型对完整的提示流程进行审查了。

> 不难发现，当前审查模型的外部挂载文档，更像是交给审查模型的“岗位职责手册”，每次开展工作前都需要查阅一遍。

#### 1.3 审查模型效果验证

&emsp;&emsp;接下来，我们就尝试借助推理链修改.md文件作为审查模型的外部挂在文件，并通过system messages读取该信息，测试此时审查模型能否成功完成get_first_email函数的审查工作。这里仍然从wrong code文件内读取错误的get_first_email函数：

In [7]:
function_name = 'get_first_email'

In [12]:
with open('./functions/wrong code/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

需要注意，此时get_first_email代码是无法运行的：

In [29]:
wrong_code = msg['stage2'][1]['content']
wrong_code

'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n    \n    # 获取第一封邮件的 Id\n    msg_id = messages[-1]["id"]\n\n    msg = service.users().messages().get(userId=\'me\', id=msg_id).execute()\n        \n    # 获取邮件内容\n    payload = msg[\'payload\']\n    headers = payload[\'headers\']\n    \n    for header in headers:\n        name = header[\'name\']\n        if name == "Date":\n            message_date = header[\'value\']\n            # 解析获得的时间字符串\n            message_timestamp = parsedate(message_date)\n

In [30]:
exec(wrong_code)

In [31]:
get_first_email()

NameError: name 'parsedate' is not defined

然后我们尝试使用审查模型对其进行修正。这里我们先简单测试模型能否理解外部挂载文档中相关信息：

In [119]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": md_content},
    {"role": "user", "content": '请问什么是推理链？'}
  ]
)

In [120]:
response.choices[0].message['content']

'在智能邮件项目中，推理链特指将用户的需求逐步推导为python函数的过程，也就是说，每个推理链的输入都是用户需求，而推理的结果则是一个可以满足用户需求的函数。每个推理链都有三个推理环节，且都以json格式表示。'

然后我们引导该模型对此前错误的提示流程进行审查和修改：

In [122]:
msg_str = json.dumps(msg)

In [123]:
'以下是一个错误的智能邮件项目的推理链，请你按照要求对其进行修改：%s' % msg_str

'以下是一个错误的智能邮件项目的推理链，请你按照要求对其进行修改：{"stage1_CD": [{"role": "user", "content": "\\u8bf7\\u67e5\\u4e0b\\u6211\\u7684\\u90ae\\u7bb1\\u91cc\\u7b2c\\u4e00\\u5c01\\u90ae\\u4ef6\\u7684\\u6536\\u4ef6\\u65f6\\u95f4\\u548c\\u90ae\\u4ef6\\u5185\\u5bb9"}, {"role": "assistant", "content": "\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}], "stage1_CM": [{"role": "user", "content": "\\u8bf7\\u67e5\\u4e0b\\u6211\\u7684\\u90ae\\u7bb1\\u91cc\\u7b2c\\u4e00\\u5c01\\u90ae\\u4ef6\\u7684\\u6536\\u4ef6\\u65f6\\u95f4\\u548c\\u90ae\\u4ef6\\u5185\\u5bb9\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}, {"role": "assistant", "content": "\\u8bf7\\u5e2e\\u6211\\u7f16\\u5199\\u4e00\\u4e2apython\\u51fd\\u6570\\uff0c\\u7528\\u4e8e\\u67e5\\u770b\\u6211\\u7684Gmail\\u90ae\\u7bb1\\u4e2d\\u7b2c

In [125]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": md_content},
    {"role": "user", "content": '以下是一个错误的智能邮件项目的推理链，请你按照要求对其进行修改：%s' % msg_str}
  ]
)

In [126]:
modified_result = response.choices[0].message['content']
modified_result

'{"stage1_CD": [{"role": "user", "content": "\\u8bf7\\u67e5\\u4e0b\\u6211\\u7684\\u90ae\\u7bb1\\u91cc\\u7b2c\\u4e00\\u5c01\\u90ae\\u4ef6\\u7684\\u6536\\u4ef6\\u65f6\\u95f4\\u548c\\u90ae\\u4ef6\\u5185\\u5bb9"}, {"role": "assistant", "content": "\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}], "stage1_CM": [{"role": "user", "content": "\\u8bf7\\u67e5\\u4e0b\\u6211\\u7684\\u90ae\\u7bb1\\u91cc\\u7b2c\\u4e00\\u5c01\\u90ae\\u4ef6\\u7684\\u6536\\u4ef6\\u65f6\\u95f4\\u548c\\u90ae\\u4ef6\\u5185\\u5bb9\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}, {"role": "assistant", "content": "\\u8bf7\\u5e2e\\u6211\\u7f16\\u5199\\u4e00\\u4e2apython\\u51fd\\u6570\\uff0c\\u7528\\u4e8e\\u67e5\\u770b\\u6211\\u7684Gmail\\u90ae\\u7bb1\\u4e2d\\u7b2c\\u4e00\\u5c01\\u90ae\\u4ef6\\u4

接下来我们查看修正之后的提示流程和代码是否正确：

In [130]:
json.loads(modified_result)

{'stage1_CD': [{'role': 'user', 'content': '请查下我的邮箱里第一封邮件的收件时间和邮件内容'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请查下我的邮箱里第一封邮件的收件时间和邮件内容当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中第一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含第一封邮件收件时间和邮件内容的对象，返回结果本身必须是一个json格式对象，且信息应该指明函数的功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中第一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含第一封邮件收件时间和邮件内容的对象，返回结果本身必须是一个json格式对象，且信息应该包含邮件收件时间和邮件内容的对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID

In [136]:
code = json.loads(modified_result)['stage2'][1]['content']

In [137]:
code

'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [22]:
exec(code)

In [28]:
get_first_email()

'{"id": "189a22b5b8bd3b02", "threadId": "189a22b5b8bd3b02", "labelIds": ["UNREAD", "IMPORTANT", "CATEGORY_PERSONAL", "INBOX"], "snippet": "\\u4f60\\u597d", "payload": {"partId": "", "mimeType": "multipart/alternative", "filename": "", "headers": [{"name": "Delivered-To", "value": "ksken166@gmail.com"}, {"name": "Received", "value": "by 2002:a05:7022:391:b0:68:fa06:8e83 with SMTP id 17csp1296694dlc;        Sat, 29 Jul 2023 08:01:38 -0700 (PDT)"}, {"name": "X-Google-Smtp-Source", "value": "APBJJlGBmPj+ot/fli8NnuhSR3GDRBjgbng8hkqdYdS46UgKqx4+1DBhrME+djgWIEAxRk4DoDjd"}, {"name": "X-Received", "value": "by 2002:a17:90b:3ec7:b0:268:6838:ec53 with SMTP id rm7-20020a17090b3ec700b002686838ec53mr4811661pjb.1.1690642897871;        Sat, 29 Jul 2023 08:01:37 -0700 (PDT)"}, {"name": "ARC-Seal", "value": "i=1; a=rsa-sha256; t=1690642897; cv=none;        d=google.com; s=arc-20160816;        b=hhau33ELBfWIJyz/vfougtEo3femzAIamahQGIoDbtImxBWwxKR2EBB6ozMi9Qn66i         dSDcI5ZR595CrECf1KlxSfJRp8xewF7DCQ2

能够发现，经过审查修改之后的模型，提示示例及最终修改之后的代码都通过了人工检验，也说明审查模型本身是具备审查效力的，当然也侧面说明此前我们编写的外部挂载文档切实有效。

&emsp;&emsp;而纵观整个审查模型的创建和使用过程不难发现，其核心难点就在于如何编写外部挂载文档来给与模型正确的引导。其实总的来说，如何用好的大语言模型，是大语言模型工程师的核心能力之一。而依托外部挂载文档更好的引导模型完成某项工作，就是用好的语言模型的关键手段，而在这个过程中，如何设计外部挂载文档，其实是最核心的、同时也是最复杂的工作事项。其复杂之处并不在于文字的编写，而在于需要在理解大语言模型本身能力的基础之上，创造力的编写能够有效激发和引导大语言模型的文字，就类似于提示流程的设计和提示词的编写，都是极具创造力的工作。而这部分内容，也是教学课程非常重要的一部分内容。

### 2.基于大语言模型的自动debug函数

#### 2.1 自动debug函数创建过程

- 审查函数的封装和创建过程

&emsp;&emsp;在完成了审查模型的创建之后，接下来，我们即可将上述审查模型的审查流程封装为一个完整的函数，方便后续直接调用。这里考虑到大语言模型输出的结果有可能markdown格式对象，因此我们需要单独定义一个函数用于从markdown格式对象中提取json对象，函数如下：

In [109]:
import re

def extract_json(s):
    pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
    match = re.search(pattern, s, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return s

然后即可测试能否从md_content中提取json对象：

In [104]:
md_content

'1.什么是智能邮件项目？智能邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮件操作，如查阅邮件、收发邮件等。项目采用的邮箱API是Gmail API，并且已经获得了授权，授权文件为本地的token.json文件。\n\n2.什么是推理链？在智能邮件项目中，推理链特指将用户的需求逐步推导为python函数的过程，也就是说，每个推理链的输入都是用户需求，而推理的结果则是一个可以满足用户需求的函数。每个推理链都有三个推理环节，且都以json格式表示，例如，以下是一个正确的推理链A：\n\n```json\n{"stage1_CD": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bGmail\\u90ae\\u7bb1\\u91cc\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002"}, {"role": "assistant", "content": "\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}], "stage1_CM": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bGmail\\u90ae\\u7bb1\\u91cc\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}, {"role": "assistant", "co

In [110]:
extract_json(md_content)

'{"stage1_CD": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bGmail\\u90ae\\u7bb1\\u91cc\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002"}, {"role": "assistant", "content": "\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}], "stage1_CM": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bGmail\\u90ae\\u7bb1\\u91cc\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}, {"role": "assistant", "content": "\\u8bf7\\u5e2e\\u6211\\u7f16\\u5199\\u4e00\\u4e2apython\\u51fd\\u6570\\uff0c\\u7528\\u4e8e\\u67e5\\u770b\\u6211\\u7684Gmail\\u90ae\\u7bb1\\u4e2d\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u4fe1\\u606f\\uff0c\\u51fd\\u6570\\u8981\\u6c42\\u5982\\u4e0b\\uff1a 

In [107]:
json.loads(extract_json(md_content))

{'stage1_CD': [{'role': 'user', 'content': '请帮我查下Gmail邮箱里最后一封邮件内容。'},
  {'role': 'assistant', 'content': '当前需求中可以作为函数参数的是：1.查看谁的邮箱。'}],
 'stage1_CM': [{'role': 'user',
   'content': '请帮我查下Gmail邮箱里最后一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮箱。'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，用于查看我的Gmail邮箱中最后一封邮件信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数内；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"},
  {'role': 'assistant',
   'content': 'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n 

接下来即可完整定义一个大语言模型审查函数：

In [2]:
def prompt_modified(function_name, system_content='推理链修改.md', model="gpt-4-0613", g=globals()):
    """
    智能邮件项目的外部函数审查函数，用于审查外部函数创建流程提示是否正确以及最终创建的代码是否正确
    :param function_name: 必要参数，字符串类型，表示审查对象名称；
    :param system_content: 可选参数，默认取值为字符串推理链修改.md，表示此时审查函数外部挂载文档名称，需要是markdwon格式文档；
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4-0613；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :return：审查结束后新创建的函数名称
    """
    print("正在执行审查函数，审查对象：%s" % function_name)
    with open(system_content, 'r', encoding='utf-8') as f:
        md_content = f.read()
        
    # 读取原函数全部提示内容
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    
    # 将其保存为字符串
    msg_str = json.dumps(msg)
    
    # 进行审查
    response = openai.ChatCompletion.create(
                    model=model,
                    messages=[
                    {"role": "system", "content": md_content},
                    {"role": "user", "content": '以下是一个错误的智能邮件项目的推理链，请你按照要求对其进行修改：%s' % msg_str}
                    ]
                )
    
    modified_result = response.choices[0].message['content']
    
    def extract_json(s):
        pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
        match = re.search(pattern, s, re.DOTALL)
        if match:
            return match.group(1)
        else:
            return s
    
    modified_json = extract_json(modified_result)
    
    # 提取函数源码
    code = json.loads(modified_json)['stage2'][1]['content']
    
    # 提取函数名
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    print("审查结束，新的函数名称为：%s。\n正在运行该函数定义过程，并保存函数源码与prompt" % function_name)
    
    exec(code, g)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # 写入函数
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
        f.write(code)
        
    # 写入提示
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(json.loads(modified_json), f)
    
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print("%s函数已在当前操作空间定义，可以进行效果测试" % function_name)
    
    return function_name

> 这里需要注意的是，对于审查函数而言，不仅可以审查提示流程是否合理、代码能否运行，如果有必要的话，也可以围绕代码风格和代码合规性进行审查。

接下来我们检测该函数能否正常执行审查任务。需要注意的是，该函数定义时是默认审查untested文件夹内的函数，这里我们将此前创建的错误函数get_first_email函数文件夹放入untested文件夹内，然后直接使用prompt_modified函数进行测试，测试结果如下：

In [80]:
prompt_modified(function_name='get_first_email', model="gpt-4-0613", g=globals())

正在执行审查函数，审查对象：get_first_email
审查结束，新的函数名称为：get_first_email。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_first_email/get_first_email_prompt.json文件中
get_first_email函数已在当前操作空间定义，可以进行效果测试


'get_first_email'

In [84]:
get_first_email?

Signature: get_first_email(userId='me')
Docstring:
查询Gmail邮箱中第一封邮件信息
:param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，    注意，当查询我的邮箱时，userId需要输入'me'；
:return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式
File:      Dynamically generated function. No source code available.
Type:      function

In [85]:
get_first_email()

'{"id": "189a22b5b8bd3b02", "threadId": "189a22b5b8bd3b02", "labelIds": ["UNREAD", "IMPORTANT", "CATEGORY_PERSONAL", "INBOX"], "snippet": "\\u4f60\\u597d", "payload": {"partId": "", "mimeType": "multipart/alternative", "filename": "", "headers": [{"name": "Delivered-To", "value": "ksken166@gmail.com"}, {"name": "Received", "value": "by 2002:a05:7022:391:b0:68:fa06:8e83 with SMTP id 17csp1296694dlc;        Sat, 29 Jul 2023 08:01:38 -0700 (PDT)"}, {"name": "X-Google-Smtp-Source", "value": "APBJJlGBmPj+ot/fli8NnuhSR3GDRBjgbng8hkqdYdS46UgKqx4+1DBhrME+djgWIEAxRk4DoDjd"}, {"name": "X-Received", "value": "by 2002:a17:90b:3ec7:b0:268:6838:ec53 with SMTP id rm7-20020a17090b3ec700b002686838ec53mr4811661pjb.1.1690642897871;        Sat, 29 Jul 2023 08:01:37 -0700 (PDT)"}, {"name": "ARC-Seal", "value": "i=1; a=rsa-sha256; t=1690642897; cv=none;        d=google.com; s=arc-20160816;        b=hhau33ELBfWIJyz/vfougtEo3femzAIamahQGIoDbtImxBWwxKR2EBB6ozMi9Qn66i         dSDcI5ZR595CrECf1KlxSfJRp8xewF7DCQ2

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1690794360312.png" alt="1690794360312" style="zoom:33%;" />

能够发现，审查函数能够顺利输出正确的审查结果。

- 自动debug函数创建过程

&emsp;&emsp;而有了审查函数之后，接下来我们即可将此前介绍的一系列debug方法进行整合，通过代码环境中的input交互方式进行自动debug。该函数名称为function_test，可以围绕外部函数可能出错的各个环节进行提示和自动修改。该函数的核心功能如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/code_generate%E5%87%BD%E6%95%B0%E5%88%9B%E5%BB%BA%E6%B5%81%E7%A8%8B.png" alt="code_generate函数创建流程" style="zoom:33%;" />

能够看出，该函数能够借助run_conversion函数的输出结果，来测试外部函数功能是否符合要求，并且集成了再次运行、重新输入用户需求和运行审查函数三种不同的debug方法，同时可以在运行的过程中实时调整Few-shot方法。并且，该函数可以直接读取文件库中相关函数和代码，同时还可以根据最终的审查结果来将新创建的外部函数从untested文件夹内转移至tested文件夹内，从而实现tested函数库的更新。

&emsp;&emsp;接下来我们就进行该函数的定义过程。这里需要注意的是，由于后续我们需要将function_test和上一小节定义的code_generate函数进行“联动”，将其封装在一个可以全自动进行函数定义和功能测试的函数中，因此function_test的参数定义和code_generate保持完全一致。该函数的定义过程如下：

In [3]:
def function_test(function_name, req, few_shot, model="gpt-4-0613", g=globals()):

    def test_messages(ueser_content):
        messages = [{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"},
                    {"role": "system", "content": "我的邮箱地址是:ksken166@gmail.com"},
                    {"role": "user", "content": ueser_content}]
        return messages
            
    messages = test_messages(req)
    
    new_function = globals()[function_name]
    functions_list = [new_function]
    
    print("根据既定用户需求req进行%s函数功能测试，请确保当该函数已经在当前操作空间定义..." % function_name)
    
    # 有可能在run_conversation环节报错
    # 若没报错，则运行：
    try:
        final_response = run_conversation(messages=messages, functions_list=functions_list, model=model)
        print("当前函数运行结果：'%s'" % final_response)
        
        feedback = input("函数功能是否满足要求 (yes/no)? ")
        if feedback.lower() == 'yes':
            print("函数功能通过测试，正在将函数写入tested文件夹")
            remove_to_tested(function_name)
            print('done')
        else:
            next_step = input("函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？")
            if next_step == '1':
                print("准备再次测试...")
                function_test(function_name, req, few_shot)
            else:
                solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
                \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
                if solution == '1':
                    # 再次运行函数创建过程
                    print("好的，正在尝试再次创建函数，请稍等...")
                    few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                    if few_shot_str == '1':
                        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        function_name = code_generate(req=req, few_shot='all', model=model, g=g)
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
                elif solution == '2':
                    # 执行审查函数
                    print("好的，执行审查函数，请稍等...")
                    function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g)
                    # 接下来带入进行测试
                    print("新函数已创建，接下来带入进行测试...")
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
                elif solution == '3':
                    new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                    few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                    if few_shot_str == '1':
                        function_name = code_generate(req=new_req, few_shot=few_shot, model=model, g=g)
                    else:
                        function_name = code_generate(req=new_req, few_shot='all', model=model, g=g)
                    function_test(function_name=function_name, req=new_req, few_shot=few_shot, g=g)
                elif solution == '4':
                    print("好的，预祝debug顺利~")
        
    # run_conversation报错时则运行：
    except Exception as e:
        next_step = input("run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？")
        if next_step == '1':
            function_test(function_name, req, few_shot)
        else:
            solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
            \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
            if solution == '1':
                # 再次运行函数创建过程
                print("好的，正在尝试再次创建函数，请稍等...")
                few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                if few_shot_str == '1':
                    function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    function_name = code_generate(req=req, few_shot='all', model=model, g=g)
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
            elif solution == '2':
                # 执行审查函数
                print("好的，执行审查函数，请稍等...")
                max_attempts = 3
                attempts = 0

                while attempts < max_attempts:
                    try:
                        function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g)
                        break  # 如果代码成功执行，跳出循环
                    except Exception as e:
                        attempts += 1  # 增加尝试次数
                        print("发生错误：", e)
                        if attempts == max_attempts:
                            print("已达到最大尝试次数，程序终止。")
                            raise  # 重新引发最后一个异常
                        else:
                            print("正在重新运行审查程序...")
                # 接下来带入进行测试
                print("新函数已创建，接下来带入进行测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
            elif solution == '3':
                new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                if few_shot_str == '1':
                    function_name = code_generate(req=new_req, few_shot=few_shot, model=model, g=g)
                else:
                    function_name = code_generate(req=new_req, few_shot='all', model=model, g=g)
                function_test(function_name=function_name, req=new_req, few_shot=few_shot, g=g)
            elif solution == '4':
                print("好的，预祝debug顺利~")

接下来我们再次读取此前出现错误的get_first_email函数并在本地进行定义，然后测试function_test函数的debug流程：

In [5]:
with open('./functions/wrong code/get_first_email/get_first_email_module.py', encoding='utf-8') as f:
    code  = f.read()

In [6]:
code

'def get_first_email(userId=\'me\'):\n    """\n    查询Gmail邮箱中第一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含第一封邮件的收件时间和邮件内容的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    \n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的全部邮件\n    results = service.users().messages().list(userId=userId).execute()\n    messages = results.get(\'messages\', [])\n    \n    # 获取第一封邮件的 Id\n    msg_id = messages[-1]["id"]\n\n    msg = service.users().messages().get(userId=\'me\', id=msg_id).execute()\n        \n    # 获取邮件内容\n    payload = msg[\'payload\']\n    headers = payload[\'headers\']\n    \n    for header in headers:\n        name = header[\'name\']\n        if name == "Date":\n            message_date = header[\'value\']\n            # 解析获得的时间字符串\n            message_timestamp = parsedate(message_date)\n

In [7]:
exec(code)

In [8]:
get_first_email()

NameError: name 'parsedate' is not defined

而接下来借助function_test进行debug的流程如下：

In [143]:
function_test(function_name = 'get_first_email', 
              req = '请帮我查下我的邮箱里面第一封邮件发件人和邮件主要内容', 
              few_shot = 'all')

根据既定用户需求req进行get_first_email函数功能测试，请确保当该函数已经在当前操作空间定义...


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 1


好的，正在尝试再次创建函数，请稍等...


准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？ 1


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_specific_email
新函数保存在./functions/untested functions/get_specific_email/get_specific_email_module.py文件中
新函数提示示例保存在./functions/untested functions/get_specific_email/get_specific_email_prompt.json文件中
done
根据既定用户需求req进行get_specific_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'你的邮箱中的第一封邮件是来自端木天的，邮箱地址为2323365771@qq.com，邮件的主要内容是"你好"。'


...

好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：get_specific_email
审查结束，新的函数名称为：get_specific_email。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_specific_email/get_specific_email_prompt.json文件中
get_specific_email函数已在当前操作空间定义，可以进行效果测试
新函数已创建，接下来带入进行测试...
根据既定用户需求req进行get_specific_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'第一封邮件的发件人是“端木天”，邮件地址为2323365771@qq.com，邮件的主要内容是“你好”。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 3
好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。 请帮我查下我的邮箱里面最早的一封邮件发件人和邮件主要内容
接下来如何运行代码创建函数？1.采用此前Few-shot方案；
2.使用全部外部函数作为Few-shot 1


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_oldest_email
新函数保存在./functions/untested functions/get_oldest_email/get_oldest_email_module.py文件中
新函数提示示例保存在./functions/untested functions/get_oldest_email/get_oldest_email_prompt.json文件中
done
根据既定用户需求req进行get_oldest_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'您的最早的一封邮件来自于“Google Payments <payments-noreply@google.com>”，邮件内容主要是告知您一个用户Georgia Kinghorne已经更改了他的Google账户的联系人电子邮件地址，付款资料 ID 为 4010-6037-4228 ，他的电子邮件地址从 2323365771@qq.com 更改为ksken166@gmail.com，并且在付款资料中我们已经更新了他的电子邮件地址以显示这一改变。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


而顺利运行完了之后，我们即可在tested文件夹内查看对应函数、代码和提示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5c1f0b0d5e62909b92cccae85dbb50c.png" alt="5c1f0b0d5e62909b92cccae85dbb50c" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/385bb75b5821d928eace2669db39e85.png" alt="385bb75b5821d928eace2669db39e85" style="zoom:33%;" />

- 全自动代码编写和debug函数：

&emsp;&emsp;最后，我们即可将上一小节定义的code_generate函数和function_test函数封装在一个函数内进行协同，即围绕某个具体的编程需求，可以在一个函数内进行代码创建和交互审查：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/code_generate%E5%87%BD%E6%95%B0%E5%88%9B%E5%BB%BA%E6%B5%81%E7%A8%8B.png" alt="code_generate函数创建流程" style="zoom:33%;" />

In [4]:
def Gmail_auto_func(req, few_shot='all', model='gpt-4-0613', g=globals(), detail=0):
    function_name = code_generate(req, few_shot=few_shot, model=model, g=g, detail=detail)
    function_test(function_name=function_name, req=req, few_shot=few_shot, model=model, g=g)

至此，我们就完成了可以自动进行编码即代码审查的Gmail_auto_func函数全部定义过程。

#### 2.2 自然语言编程性能测试

&emsp;&emsp;接下来我们围绕Gmail_auto_func进行更多的功能测试，为了更好的验证大预言模型的编程能力，以下问题多由学员随堂提出，具体问题以及模型创建代码过程如下：

- Q1.'请帮我看下我的邮箱里面，来自端木天的邮件总共有几封。'：一次完成

In [172]:
Gmail_auto_func(req = '请帮我看下我的邮箱里面，来自端木天的邮件总共有几封。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_from_sender
新函数保存在./functions/untested functions/count_emails_from_sender/count_emails_from_sender_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_from_sender/count_emails_from_sender_prompt.json文件中
done
根据既定用户需求req进行count_emails_from_sender函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'来自端木天的邮件总共有4封。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q2.'请帮我看下我的邮箱里面，来自端木天的邮件总共有几封。'：修改三次后完成

In [173]:
Gmail_auto_func(req = '请帮我看下我的邮箱里面7月28号之前总共有几封邮件。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_email_count_before_date
新函数保存在./functions/untested functions/get_email_count_before_date/get_email_count_before_date_module.py文件中
新函数提示示例保存在./functions/untested functions/get_email_count_before_date/get_email_count_before_date_prompt.json文件中
done
根据既定用户需求req进行get_email_count_before_date函数功能测试，请确保当该函数已经在当前操作空间定义...


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 1


根据既定用户需求req进行get_email_count_before_date函数功能测试，请确保当该函数已经在当前操作空间定义...


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 1


好的，正在尝试再次创建函数，请稍等...


...

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_before_date
新函数保存在./functions/untested functions/count_emails_before_date/count_emails_before_date_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_before_date/count_emails_before_date_prompt.json文件中
done
根据既定用户需求req进行count_emails_before_date函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在你的邮箱中，收件时间在2023年7月28号之前的邮件总共有0封。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：count_emails_before_date
审查结束，新的函数名称为：count_emails_before_date。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/count_emails_before_date/count_emails_before_date_prompt.json文件中
count_emails_before_date函数已在当前操作空间定义，可以进行效果测试
新函数已创建，接下来带入进行测试...
根据既定用户需求req进行count_emails_before_date函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在您的邮箱里，2023年7月28号之前收到的邮件总共有18封。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q3.'请帮我统计下我的邮箱里面总共有几封未读邮件'：一次完成

In [174]:
Gmail_auto_func(req = '请帮我统计下我的邮箱里面总共有几封未读邮件')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_unread_email_count
新函数保存在./functions/untested functions/get_unread_email_count/get_unread_email_count_module.py文件中
新函数提示示例保存在./functions/untested functions/get_unread_email_count/get_unread_email_count_prompt.json文件中
done
根据既定用户需求req进行get_unread_email_count函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'您的邮箱里总共有2封未读邮件。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q4.'请帮我统计下我的邮箱里面总共有多少封内容包含Goole单词的邮件。'：一次完成

In [175]:
Gmail_auto_func(req = '请帮我统计下我的邮箱里面总共有多少封内容包含Goole单词的邮件。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_by_keyword
新函数保存在./functions/untested functions/count_emails_by_keyword/count_emails_by_keyword_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_by_keyword/count_emails_by_keyword_prompt.json文件中
done
根据既定用户需求req进行count_emails_by_keyword函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'对不起，我没有找到包含"Goole"这个关键词的邮件。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 3
好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。 请帮我统计下我的邮箱里面总共有多少封内容包含Google单词的邮件。
接下来如何运行代码创建函数？1.采用此前Few-shot方案；
2.使用全部外部函数作为Few-shot 2


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_with_query
新函数保存在./functions/untested functions/count_emails_with_query/count_emails_with_query_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_with_query/count_emails_with_query_prompt.json文件中
done
根据既定用户需求req进行count_emails_with_query函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'您的邮箱中共有3封邮件包含"Google"这个词。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q5.'请帮我查下，我的邮箱里面，2023年7月19号到2023年7月28号之间的邮件当中，总共有几封邮件是端木天发来的邮件。'：一次修改后完成

In [176]:
Gmail_auto_func(req = '请帮我查下，我的邮箱里面，2023年7月19号到2023年7月28号之间的邮件当中，总共有几封邮件是端木天发来的邮件。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_within_range
新函数保存在./functions/untested functions/count_emails_within_range/count_emails_within_range_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_within_range/count_emails_within_range_prompt.json文件中
done
根据既定用户需求req进行count_emails_within_range函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在2023年7月19号到2023年7月28号期间，你的邮箱中没有收到端木天的邮件。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：count_emails_within_range


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 3
好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。 '查下我的邮箱里面，2023-7-19到2023-7-28之间端木天发来的邮件有几封'
接下来如何运行代码创建函数？1.采用此前Few-shot方案；
2.使用全部外部函数作为Few-shot 2


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:count_emails_from_sender_in_period
新函数保存在./functions/untested functions/count_emails_from_sender_in_period/count_emails_from_sender_in_period_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_from_sender_in_period/count_emails_from_sender_in_period_prompt.json文件中
done
根据既定用户需求req进行count_emails_from_sender_in_period函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在2023年7月19日至2023年7月28日期间，端木天发给你的邮件共有3封。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


至此，我们就完成了一个高度自动化的编程函数。同时，该流程也是目前业内大语言自动编程最完善、也是单函数编程功能最强的流程之一。接下来我们的编程共工作也将主要依赖该函数完成。

> 由此，我们也可以深度体验“自然语言编程”的威力与效率。

### 3.高阶自然语言编程：基于大模型的分段编写代码策略

#### 3.1 大模型编程的性能瓶颈

&emsp;&emsp;在完成了全自动代码编写和审查函数之后，我们已经能够引导GPT模型完成大量AI邮件系统相关代码开发工作了。但是受到大模型本身理解能力和编程能力限制，在面对一些复杂需求或者处理一些大模型本身不太容易理解的问题时（如处理时间类问题），这套流程仍然无法稳定高效的完成相关函数的代码编写工作。当然，问题越难，大模型编写代码的准确率就越低。例如面对如下需求：请帮我看下我的邮箱里面，最早的一封来自端木天的邮件是什么时候，以及邮件的核心内容是什么？在调用此前定义的Gmail_auto_func函数时，经过若干轮的调试和debug，仍然无法获得正确的结果：

In [145]:
Gmail_auto_func(req = '请帮我看下我的邮箱里面，最早的一封来自端木天的邮件是什么时候，以及邮件的核心内容是什么？')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_earliest_email_from
新函数保存在./functions/untested functions/get_earliest_email_from/get_earliest_email_from_module.py文件中
新函数提示示例保存在./functions/untested functions/get_earliest_email_from/get_earliest_email_from_prompt.json文件中
done
根据既定用户需求req进行get_earliest_email_from函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'最早的一封来自端木天的邮件是在2023年7月29日，邮件的核心内容为“你好”。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：get_earliest_email_from
审查结束，新的函数名称为：get_earliest_email_from。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_earliest_email_from/get_earliest_email_from_prompt.json文件中
get_earliest_email_from函数已在当前操作空间定义，可以进行效果测试
新函数已创建，接下来带入进行测试...
根据既定用户需求req进行get_earliest_email_from函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'很抱歉，我没有查看您个人邮箱的权限，因为这涉及到个人隐私。但是如果您想查找最早的一封来自端木天的邮件，您可以在登录邮件后，在邮件搜索框内输入他的名称或者邮箱地址，并做时间上的筛选。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 3
好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。 请帮我看下我的邮箱里面时间最早的一封来自端木天的邮件，请告诉我这封邮件的收件时间和主要内容
接下来如何运行代码创建函数？1.采用此前Few-shot方案；
2.使用全部外部函数作为Few-shot 2


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_first_email_from_sender
新函数保存在./functions/untested functions/get_first_email_from_sender/get_first_email_from_sender_module.py文件中
新函数提示示例保存在./functions/untested functions/get_first_email_from_sender/get_first_email_from_sender_prompt.json文件中
done
根据既定用户需求req进行get_first_email_from_sender函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'对不起，我没有找到来自端木天（2323365771@qq.com）的邮件。'


...

准备再次测试...
根据既定用户需求req进行get_first_email_from_sender函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'抱歉，由于隐私保护政策，我无法直接查看你的邮件内容。你可以直接登录你的邮箱查阅。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：get_first_email_from_sender
审查结束，新的函数名称为：get_latest_email。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_latest_email/get_latest_email_prompt.json文件中
get_latest_email函数已在当前操作空间定义，可以进行效果测试
新函数已创建，接下来带入进行测试...
根据既定用户需求req进行get_latest_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'端木天给您发的第一封邮件是在2023年7月29日08:01:37，邮件的内容是："你好"。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 4


好的，预祝debug顺利~


上述需求尽管看起来并不复杂，但实际上却是戳中了大模型的软肋——即大模型本身对时间这一概念理解不到位，在某些语境下往往不能很好的区分时间的先后点，即无法分清哪个时间点在前、哪个时间点再后。而对于另一类同时包含多项动作的需求，大模型的编程正确率同样不高。例如有如下需求：请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签。很明显该需求需要在一段代码中同时先识别包含开会邀请的邮件，然后再将其打上标签。

&emsp;&emsp;接下来我们测试大模型能否一步到位编写能够实现该需求的函数。为了实现该需求，我们首先需要在当前的授权token中再加上邮件修改以及邮箱的标签获取相关权限，这里我们再次执行下列代码进行授权：

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send','https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/gmail.labels']
flow = InstalledAppFlow.from_client_secrets_file(
                'client_web1.json', SCOPES)
creds = flow.run_local_server(port=8899, access_type='offline', prompt='consent')

with open('token.json', 'w') as token:
    token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=597598790553-j4tj274ajpeta15s9lc0sffmlqmbhi9q.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8899%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.labels&state=rxcRICv5s8cjEVFF1KwEwJuPx1JBnH&access_type=offline&prompt=consent


然后需要在Gmail邮箱里添加一个会议预定的标签：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d439866b5c5a23d04925db388c9882c.png" alt="d439866b5c5a23d04925db388c9882c" style="zoom:33%;" />

同时，当前邮箱里面也接收到了一封有关会议邀请的邮件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1201c7b64edbf80223104022f707985.png" alt="1201c7b64edbf80223104022f707985" style="zoom:33%;" />

接下来，我们调用Gmail_auto_func来编写满足该需求的代码：

In [3]:
Gmail_auto_func(req = '请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:label_unread_emails_by_keyword
新函数保存在./functions/untested functions/label_unread_emails_by_keyword/label_unread_emails_by_keyword_module.py文件中
新函数提示示例保存在./functions/untested functions/label_unread_emails_by_keyword/label_unread_emails_by_keyword_prompt.json文件中
done
根据既定用户需求req进行label_unread_emails_by_keyword函数功能测试，请确保当该函数已经在当前操作空间定义...
An error occurred: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/ksken166%40gmail.com/messages/189bfa28e31ab424/modify?alt=json returned "Invalid label: 会议预定". Details: "[{'message': 'Invalid label: 会议预定', 'domain': 'global', 'reason': 'invalidArgument'}]">
当前函数运行结果：'很抱歉，出现了一个错误。看起来添加的标签 "会议预定" 是无效的，请您确认此标签的正确性，并再次尝试。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：label_unread_emails_by_keyword


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 1


好的，正在尝试再次创建函数，请稍等...


准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？ 1


第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:label_unread_emails
新函数保存在./functions/untested functions/label_unread_emails/label_unread_emails_module.py文件中
新函数提示示例保存在./functions/untested functions/label_unread_emails/label_unread_emails_prompt.json文件中
done
根据既定用户需求req进行label_unread_emails函数功能测试，请确保当该函数已经在当前操作空间定义...


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 4


好的，预祝debug顺利~


能够发现，经过多轮的调试和debug，仍然无法获得一个准确的编程结果，邮箱里面的会议邀请邮件仍然没有被打上标签。

> 需要注意的是，由于大模型编程本身存在一定的不稳定性，因此这里列举的两个需求示例在某些情况下也是能够被编程成功的，但相比于其他简单需求，大模型对于复杂需求的编程准确性明显大幅下降。

&emsp;&emsp;经过上述验证不难发现，尽管当前的大模型编程流程已经添加了非常多的提高稳定性的措施（诸如多段提示、提示流程审查和代码审查等），但对于复杂的编程任务，大模型编程的准确率仍然很低。面对这一问题，有什么解决方案呢？需要注意的是，对于大模型工程师来说，如何“用好”大模型始终是岗位核心竞争力，而自然语言编程又是大模型非常重要的应用领域，因此，更进一步的思考和探讨如何进一步提高大模型对复杂问题的编程能力，是非常有必要的。

#### 3.2 编程性能瓶颈根源与解决方案

&emsp;&emsp;首先我们需要明确大模型编程性能瓶颈的根源，以及据此来探讨进一步提升编程性能的方法。

- 编程性能瓶颈根源

&emsp;&emsp;其实大模型对复杂问题的编程能力不足，和大模型对于创作长文本能力不足的原因是一样的——都是因为模型本身并不具备“长期记忆”，而这会导致大模型无法一直围绕某个恒定的目标进行长段的文本生成，需要注意的是，对于大模型来说，代码编写本质上就是按照某种特定格式进行文本创作，本质上并没有区别，而面对复杂需求，往往意味着为了实现该需求需要编写长段的代码，因此受限于大模型编写长段代码能力不足，对于复杂需求的代码编写准确性也随之降低。

- 提升长段代码编程稳定性策略

&emsp;&emsp;而要如何解决这个问题呢？首先需要明确的是，此时大模型所表现出来的问题，并不是需求理解不足，而是在充分理解需求的情况下，也无法编写长段代码来满足需求，也就是说大模型在此前的LtM提示流程下是能够充分理解用户需求的，也能够顺利的辨别需求背后的变量（作为函数参数），问题在于实现该需求层面代码本身编写能力不足。因此我们解决问题的重点是如何提高模型的编程能力（而不是通过提示工程来进一步提升模型理解能力。）。

&emsp;&emsp;那么应该如何提升大模型的编写长段代码的稳定性呢？和编写长文本类似，最简单有效的方法就是先列提纲，然后分段编写，最后进行合并，即通过分段完成的方式，来提高模型长文本（长段代码）编写的表现。当然，编程任务还有其特殊之处，例如分段编写的代码需要变量一致，并且前后代码功能需要很好的衔接，此外我们已经编写了非常多辅助代码编写的函数，还需要考虑尽可能借助这些辅助函数来提高开发效率等。而课程截止目前，我们也已经积累了较为丰富的大语言模型相关开发流程的设计经验，并且也具备了一定的“调教”大模型的经验，接下来我们在“分段编写再合并”这一基本思路指导下，先尝试头脑风暴，尽可能列举理论上可行的实现方案，然后再选择其一进行落地代码实现。

- 第一阶段：复杂需求拆解

&emsp;&emsp;既然是要分段编写代码再进行合并，首先就需要先对原始的复杂需求进行拆解。这个拆解的过程就非常类似于CoT思考链——一步步思考并解决问题。例如对于需求：请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签。就可以拆分为步，第一步是找到所有未读邮件，第二步是在未读邮件中找到有关会议邀请的邮件，并将其添加会议预定的标签。不难看出，这一拆分过程本身并不复杂，甚至相比CoT，围绕当前Gmail智能邮件项目的复杂任务拆分流程会更加具备确定性，因此也会更加简单。

> 需要注意的是，严格意义来说，CoT或者LtM拆分得到的子任务往往不是确定的任务，CoT和LtM更多其实是一种引导模型进行思考和处理的方式，而引导的结果往往具备一定的不确定性。

&emsp;&emsp;这里所谓的拆分任务的确定性，主要是因为拆分得到的子任务往往是一些Gmail API的一些最小功能单位，比如查找邮件、给邮件打标签等。而一旦拆分的子任务属于较为确定性的一类任务，那么这个拆分任务这一过程本身，其实就是一类特殊且简单的拆分任务。而要引导大语言模型完成特殊且较为简单的任务，最好的方式就是输入一些项目背景信息（挂载外部文档）并且提供一些拆解的示例作为Few-shot。

&emsp;&emsp;这里需要注意的是，在进行进行子任务时，必须强调子任务需要是Gmail API能够简单实现的子任务，否则模型在进行子任务拆分时甚至会认为第一步是打开邮箱，而另一方面，只有拆分的子任务是Gmail API能够简单快速实现的任务，才能尽可能确保子任务对应的代码编写的准确性。而根据我们对大模型性能的了解，这方面信息的顺利传递，不能只通过Few-shot，还必须依靠编写一个外部文档进行项目说明。此外需要注意的是，这里如果找不到合适的任务拆分示例作为Few-shot，也可以在充分描述背景之后让大模型帮我们进行编写。

&emsp;&emsp;而当我们已经完成任务拆分之后，接下来就是考虑如何围绕子任务进行代码编写，以及各部分代码如何拼接等问题了。这些工作我们统一称为复杂任务编程的第二阶段，第二个阶段的实现方法可以有很多种，我们这里依次对其进行介绍：

- 第二阶段方案一：单独创建复杂需求函数

&emsp;&emsp;首先最容易想到的方法，就是在基础已有的基础功能函数基础之上再架设一层高层函数，用于组合基础函数功能，来实现更加复杂的需求。这里所谓的基础功能函数，指的是可以借助code_generate函数进行准确高效编写的基础功能函数，例如此前我们创建的查看未读邮件、发送邮件、邮件汇总等功能函数，而从任务拆分角度来看，这些基础函数就是专门用于去实现子任务的相关函数。而在面对复杂需求时，我们可以围绕拆分出来的子任务先分别编写对应的功能实现函数，然后再编写一个可以调用基础函数的高层函数，用于将基础函数的输出结果进行整合，并最终完成原始复杂任务的处理。

例如对于需求：请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签。就可以拆分为步，第一步是找到所有未读邮件，第二步是在未读邮件中找到有关会议邀请的邮件，并将其添加会议预定的标签。

&emsp;&emsp;例如，对于当前需求“请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签”（以下简称为复杂需求或者当前需求）来说，我们已经将其拆分为了两个子任务，分别是获取未读邮件列表（以下简称为子任务1），以及从未读邮件中挑选会议相关邮件，并为其添加会议预定标签（以下简称为子任务2）。这两个子任务可以分别借助此前定义好的code_generate函数+function_test函数进行创建，同时需要注意在创建完成子任务2的函数（以下简称为函数2）时，需要确保其输入的参数格式和完成子任务1的函数（以下简称为函数1）的输出结果一致。而当我们创建完函数1和函数2之后，接下来即可定义一个外部函数，用于串联函数1和函数2来进行运行，即可获得最终结果。

&emsp;&emsp;具体方案一的执行流程可以通过如下流程图进行表示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E5%A4%8D%E6%9D%82%E9%9C%80%E6%B1%82%E5%87%BD%E6%95%B0%E7%BC%96%E5%86%99%E6%96%B9%E6%A1%88%E4%B8%801.png" alt="复杂需求函数编写方案一1" style="zoom:33%;" />

&emsp;&emsp;不难发现，方案一的这种分层开发的思路其实非常像传统技术团队的研发策略，这种策略便于分工、易于管理，并且有非常明显的阶段性产出，同时如果是人工进行代码编写，也能非常便捷的维持函数1的输出结果和函数2的参数一致。并且，这种开发方案的优势在于代码利用率很高，试想一下，伴随着基础函数库的函数功能不断丰富，会有越来越多的复杂需求或许并不需要单独编写子任务函数，而可以直接从已有的函数库中挑选函数来组合运行。例如，如果现在的需求是在未读邮件中寻找会议相关邮件，并将邮件内容编写为附件本地保存，则在分解任务时，分解得到的第一个子任务——列举未读邮件列表，就可以直接从利用函数1来完成。总的来说，方案一是一种易于管理且代码利用率极高的一个方案。

&emsp;&emsp;但是，该方案也存在一些问题，其中最核心的问题就在于现在是引导大模型进行代码开发，而不是引导人进行代码开发。这其中会存在较为本质的区别，对于程序员来说，能够非常简单的的统一一系列函数的输出和输入格式，但如果是自然语言编程，这一点就很难做到，我们很难向大模型传递“函数1的输出结果要用于函数2，因此函数1需要输出每一封未读邮件的邮件ID，否则函数2就无法定位具体的邮件”。因此，在实际执行方案一时，哪怕函数1和函数2都能顺利编写代码，但二者也很难顺利衔接到一起来进行运行。

&emsp;&emsp;此外，对于大语言模型来说，哪怕对复杂问题进行了拆解，并且子任务拥有可以完成对应需求的基础函数，大语言模型也很难识别当前子任务可以用该函数。这个问题主要源于大语言模型对语义理解存在不稳定性，哪怕每个基础函数都拥有一套详细的函数说明（大语言模型自己编写的函数说明），但这些不稳定性会对高层函数编写的准确性造成非常大的影响。有趣的是，这个问题其实完全不会对人类程序员造成任何影响，因此这也是我们在考虑方案一是否可行时往往会被忽视的一个问题。

&emsp;&emsp;其实对于大多数初级大模型工程师来说，都习惯以人的思维思考大模型的运行状态，而实际上，要设计一套引导大模型来执行任务的流程，就必须优先考虑到大模型和人的区别，很多看起来对人类程序员非常友好的流程，却不一定适用于引导大模型来完成。一个为大模型设计的任务流程需要充分发挥大模型的优势，而不能先入为主的优先从人的角度来考虑问题。

> 这里需要注意，关于理解函数功能相关语义、以及统一部分函数输入和输出结果一致性等问题，可以通过知识图谱技术手段来解决。相关内容超出本次课程讨论范畴，感兴趣的同学可以自行查阅资料，并借助知识图谱技术来构建一套稳定可执行的方案一流程。

&emsp;&emsp;尽管大模型在函数功能的语义理解层面不如人类程序员，但大模型却有自己的得天独厚的优势——能够不知疲倦的反复高效的创建代码，并且能够对自己编写的代码进行自动审查。因此我们完全可以利用这点，来创建一个更适合大模型来执行的复杂需求函数编写流程。因此，接下来的两个方案不再考虑拆分创建个函数再组合运行的思路，而是考虑采用一个函数来满足复杂需求，并且充分借助大模型能够进行自我审查的特性，来提高这一个函数对需求满足的准确性。

- 第二阶段方案二：创建嵌套函数

&emsp;&emsp;而如何在一个函数内完成复杂需求功能设计，首先能想到的就是创建嵌套函数，即将方案一中的函数1作为内部函数，函数2作为外部函数来创建一个嵌套函数，并由此来满足原始需求。具体执行流程如下：首先我们先单独引导大模型编写函数1和函数2的需求，并且在提示中规定函数1的输出将作为函数2的输入，然后先创建函数2，在创建完函数2的代码之后，再将函数2的参数中原本用于表示函数1输出的参数，改为函数1的参数，然后再将函数2的代码作为提示，并且在提示中说明函数1和2的功能，以及函数1是函数2的内部函数等信息，以此引导大模型创建函数1。此时由于模型是在知道函数1和2之间的关系、以及函数2的代码基础之上创建的函数1，便能够自动判断需要将未读邮件的邮件ID作为函数1的输出，才能让函数2顺利的为某些函数打上标签。在创建完函数1和函数2之后，再将其进行合并（函数1作为函数2的内部函数），并输出最终结果。

&emsp;&emsp;具体方案二的执行流程可以通过如下流程图进行表示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E5%A4%8D%E6%9D%82%E9%9C%80%E6%B1%82%E5%87%BD%E6%95%B0%E7%BC%96%E5%86%99%E6%96%B9%E6%A1%88%E4%BA%8C.png" alt="复杂需求函数编写方案二" style="zoom:33%;" />

&emsp;&emsp;很明显，该流程能够非常好的借助大模型本身的能力，解决此前所面临的函数1和2无法很好的协调输入和输入的问题，同时由于一个复杂需求一个函数，因此也不存在方案一中由于大模型语义理解不稳定性造成潜在风险。总的来说，相比方案一，方案二可以说是能够更好的发挥大模型本身的能力。但是，通过仔细观察上述方案流程不难发现，方案二最核心的问题就在于需要反复给与大模型非常多段的提示（例如函数2的参数修改、函数1和2的关系等），才能很好的引导大模型完成相关工作。尽管我们也可以通过创建某个函数来流程化的批量执行这些提示，但这个流程和此前我们创建的用于编写基础函数的流程差距太大，因此并不能最大程度借鉴此前流程，从而导致需要耗费大量的人工成本、且整体效率较低。

&emsp;&emsp;介于此，我们需要在方案二的基础之上，进一步优化整个流程，即在这个复杂函数自动编写流程中，不仅需要扬长避短发挥大模型特性，同时也需要尽可能做到简洁优雅，需要尽可能借用此前基础函数的开发流程，并且保证一个较高的准确性。

- 第二阶段方案三：分别创建函数再统一代码

&emsp;&emsp;而最后一个方案则是在前两个方案基础之上进行优化和迭代的最终版方案，该方案非常不符合人类程序员的一般开发思路，但却是非常适合大语言模型来执行的开发流程，并且能够非常好的复用此前的基础函数的提示和审查流程，从而极大程度提升开发效率。

&emsp;&emsp;方案三执行流程如下，首先和方案一一样，都是拆分两个子任务，并且分别编写两个函数以满足这两个子任务需求。并且在创建函数1和2时也是采用code_generate+function_test来进行代码编写和审查，但这里需要注意，我们在创建这两个函数时，会先创建函数1，然后将函数的输出结果作为函数2创建时的需求的一部分（注意，这里只涉及需求的拼接，不涉及提示流程的修改）。并且，我们在创建函数1和函数2时，只要求代码通顺即可，并不要求两个函数能够拼凑成一个完整的链路，即不要求函数1的输出结果是函数2的输入。此时根据我们对大模型编程能力的判断，大概率函数1和2能够分别顺利运行，但无法组成一个完整的链路。但没有关系，接下来我们再次调用code_generate+function_test流程来围绕原始需求进行函数编写（以下将该函数称为原始需求函数），并且在这个过程中同时将函数1和2的源码作为system_message带入这个编写过程，此时，大模型有函数1和2的代码作为参考，即可非常顺利的完成原始需求函数编写。需要注意的是，原始需求函数并不是函数1和函数2的拼接，而是在参考了函数1和2的代码基础之上，重新编写的一个独立且完整的函数，因此哪怕函数1和2本身并不能很好的进行衔接，也不会影响原始需求函数的代码正确性。并且，此时有了函数1和2的代码作为参考，大模型围绕复杂需求进行编程的准确性也将大幅提升。更重要的是，这个过程基本是完全复用code_generate+function_test流程，无需进行更复杂大模型提示即可完成，开发效率很高。

&emsp;&emsp;具体方案三执行流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E5%A4%8D%E6%9D%82%E9%9C%80%E6%B1%82%E5%87%BD%E6%95%B0%E7%BC%96%E5%86%99%E6%96%B9%E6%A1%88%E4%B8%891.png" alt="复杂需求函数编写方案三1" style="zoom:33%;" />

&emsp;&emsp;当然，对于这个流程而言，从人类程序员的角度来看可能存在代码利用率较低，且代码生产的中间过程较长等问题。但是很明显，这些问题对于大模型来说，其实都不是问题。相比人类程序员，我们在借助大模型进行代码编写时，大模型创建单独一段代码的效率很高，因此只要最终能创建一个满足要求的函数，基本不用考虑代码复用率的问题。并且，大模型审查和学习其他代码的能力很强，在上述流程中，当我们已经完成了函数1和2的创建之后，哪怕是简单的输入原始需求、并且让大模型整合函数1和2的代码（提示如下图所示），也可以相对较为准确的创建复杂需求函数。而这一点对于人类程序员来说，会非常困难（可以试想下让你根据某项需求整理别人的多段代码，是多么低效的事情）。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/d02ce3f46e29b9f7aa698cc730e5e62.png" alt="d02ce3f46e29b9f7aa698cc730e5e62" style="zoom:33%;" />

同时，由于最后存在一个代码审查和重写的流程，并且在没有验证前后函数是否能够顺利衔接的情况下，我们将函数1的输出作为函数2的输入进行提示来创建函数2，可能也无法真正验证函数2是否可行。因此我们甚至可以在创建子函数时直接放弃测试环节，只让其进行函数创建即可，即只采用code_generate进行函数创建。由此方案三的执行流程也可以精简为如下形式：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E6%96%B9%E6%A1%88%E4%B8%89%E7%AE%80%E5%8C%96%E7%89%88.png" alt="方案三简化版" style="zoom:33%;" />

&emsp;&emsp;同时，方案三的函数编写流程也同时具备非常好的可拓展性，例如面对某项复杂需求拆分成3个甚至是更多个子任务的情况，上述流程也能非常好的应对——只需要按照流程创建3个子函数，然后再让大模型进行拼接即可。从这个角度来说，方案三更像是先让大模型自己为自己添加一些辅助代码，然后再据此创建原始需求函数。

&emsp;&emsp;综合来看，方案三是一个非常适合大模型来进行复杂问题编程的流程，不仅能够非常好的发挥大模型编程效率高且善于阅读和审查程序的编程特性，而且能够最大程度复用此前设计的code_generate+function_test代码编写流程，更重要的是，该流程的很多环节都有审查和自动debug流程，能够非常好的确保最终输出一个能够满足需求的函数。接下来，我们就尝试按照方案三的流程来创建复杂需求函数，并尝试将整个流程封装为一个更高层的函数，便于之后进行快速调用。当然，也建议大家课后可以尝试手动实现方案一和方案二，将非常有助于提升大模型应用能力，同时通过不同方案的对比，也能够更加深刻的体会什么是“用好大模型”，以及如何才能“用好大模型”。

> 再次强调，如何“用好”大模型，是大模型工程师的核心能力。同样的模型，不同的使用方法将带来完全不同的结果。

> 自然语言编程的核心，是需要通过设计某个流程来对冲大语言模型本身语义理解的不稳定性，来最终获得一个稳定的编程结果。

#### 3.3 复杂需求的自然语言编程流程

&emsp;&emsp;在进行了一整套完整的理论分析之后，接下来我们尝试手动实现方案三。

- Stage 1.复杂需求拆解

&emsp;&emsp;首先，我们需要创建一个专门用于复杂问题拆解的模型。根据此前的分析，该模型同时需要外部挂载文档以说明项目背景和复杂任务拆解的目的，同时还需要一些复杂任务拆解的示例作为Few-shot。首先先看外部文档编写方法，这里我们可以借鉴此前的《推理链修改》文档，要求非常清晰的说明项目背景，同时需要尽可能借助大模型本身的知识、通过类比的方法来介绍任务拆解的目的，并且还需要在文档的末尾进行身份设定。我们将上述内容统一编写到《复杂任务拆解》文档内，文档内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1691245639843.png" alt="1691245639843" style="zoom:33%;" />

在这里我们对模型进行了5个方面的信息输入，首先是关于当前邮件项目的项目简介，第二点是关于任务拆解的介绍，需要注意的是，由于大模型本身就了解任务拆解的基本概念，因此第二点重点强调关于智能邮件项目的任务拆解流程；第三点是定义最小子任务概念，由于在当前复杂任务拆解过程中，我们需要确保拆解的子任务是非常容易完成的（对应函数是较为容易进行编写的），因此这里通过最小子任务的概念来介绍最终拆解的目标；第四点是关于拆解的注意事项，主要是任务顺序问题；而最后一点则是进行身份设定，规定模型身份以及对应的工作事项。

&emsp;&emsp;接下来，即可采用如下方式将其读取为一个字符串格式对象：

In [55]:
with open('复杂任务拆解.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
print(md_content)

1.什么是智能邮件项目？智能邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮件操作，如查阅邮件、收发邮件等。项目采用的邮箱API是Gmail API，并且已经获得了授权，授权文件为本地的token.json文件；

2.什么是任务拆解？在智能邮件项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和端木天之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；

3.什么是最小子任务？最小子任务指的是在在智能邮件项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和端木天之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以借助Gmail API来快速的实现的功能；

4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。

5.你是一名任务拆解助手，负责智能邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。


后续即可将其作为system message进行输入。

&emsp;&emsp;接下来继续创建Few-shot的提示示例。这里总共编写了四组难度和复杂度各不相同的四个需求进行拆解，具体提示示例如下：

In [56]:
example1_user = 'Q:请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结。'
example1_assistent = 'A:1.请帮我整理端木天发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'
example2_user = 'Q:请帮我查下最近一封未读邮件的发件人。'
example2_assistent = 'A:1.请帮我查下最近一封未读邮件的发件人。'
example3_user = 'Q:请分析我邮箱里全部已收到的邮件，告诉我最常联系的人。'
example3_assistent = 'A:1.请帮我整理邮箱里全部已收到邮件的邮件列表。2.根据某份邮件列表，分析并找出最常联系的人。'
example4_user = 'Q:请在我邮箱里查找8月5号到6号的未读邮件，并找出工作相关的未读邮件，请依次给这些未读邮件回复说我正在休假，同时请将这些邮件转发给我的助理端木天。'
example4_assistent = 'A:1.请查找并整理我邮箱里8月5号到6号的未读邮件列表。2.根据某份邮件列表，查找并汇总其中和工作有关的油价你列表。3.根据某份邮件列表，对其逐一进行回复，告诉他们我正在休假。4.根据某份邮件列表，将其逐一转发给我的助手端木天。'

需要注意的是，这里我们统一通过Q&A的方式来提示模型不同内容的类型，同时拆解之后的每段文字也都以句号结尾，方便后续进行结构化字符串提取。在创建的四个提示实力中，1、3示例属于一般示例，对于大多数复杂需求来说，拆解为两个子任务基本就基本能够构成较好的自然语言编程引导，而第二个提示示例则是提醒模型，当需求较为简单时，不用进行需求拆解；而最后一个提示的示例，则是一个相对来说非常复杂的需求，总共拆解得到了4个子任务，是为了提醒模型在必要时，可以多拆分一些子任务。

> 这里其实也可以将《复杂任务拆解》文档输入给大模型，让模型先帮我们创建一些示例，然后再进行人工校验和筛选。

&emsp;&emsp;最后，我们尝试将system message和Few-shot输入给模型，并让模型围绕当前需求进行拆解，目前需求为：

In [57]:
req = '请查阅我邮箱里面未读邮件，从挑选有关会议邀请的邮件，并将其添加一个名称为会议预定的标签'

系统消息为：

In [58]:
system_message = [{"role": "system", "content": md_content}]

Few-shot为：

In [59]:
decomp_few_shot = [{"role": "user", "content": example1_user},
                   {"role": "assistant", "content": example1_assistent}, 
                   {"role": "user", "content": example2_user}, 
                   {"role": "assistant", "content": example2_assistent}, 
                   {"role": "user", "content": example3_user}, 
                   {"role": "assistant", "content": example3_assistent}, 
                   {"role": "user", "content": example4_user}, 
                   {"role": "assistant", "content": example4_assistent}]

接下来将其组合为一个完整的messages：

In [60]:
messages = system_message + decomp_few_shot
messages

[{'role': 'system',
  'content': '1.什么是智能邮件项目？智能邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮件操作，如查阅邮件、收发邮件等。项目采用的邮箱API是Gmail API，并且已经获得了授权，授权文件为本地的token.json文件；\n\n2.什么是任务拆解？在智能邮件项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和端木天之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；\n\n3.什么是最小子任务？最小子任务指的是在在智能邮件项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和端木天之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以借助Gmail API来快速的实现的功能；\n\n4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。\n\n5.你是一名任务拆解助手，负责智能邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。'},
 {'role': 'user', 'content': 'Q:请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结。'},
 {'role': 'assistant', 'content': 'A:1.请帮我整理端木天发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'},
 {'role': 'user', 'content': 'Q:请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'assistant', 'content': 'A:1.请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'user', 'content': 'Q:请请分析我邮箱里全部已收到的邮件，告诉我最

这里需要注意，为了更好的重复调用上述messages，我们将其保存在decompose_messages.json文件中，并将其放在tested functions内：

In [61]:
with open('./functions/tested functions/%s.json' % 'decompose_messages', 'w') as f:
    json.dump(messages, f)

接下来输入用户需求：

In [132]:
messages.append({"role": "user", "content": req})

In [133]:
messages

[{'role': 'system',
  'content': '1.什么是智能邮件项目？智能邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮件操作，如查阅邮件、收发邮件等。项目采用的邮箱API是Gmail API，并且已经获得了授权，授权文件为本地的token.json文件；\n\n2.什么是任务拆解？在智能邮件项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和端木天之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；\n\n3.什么是最小子任务？最小子任务指的是在在智能邮件项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和端木天之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以借助Gmail API来快速的实现的功能；\n\n4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。\n\n5.你是一名任务拆解助手，负责智能邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。'},
 {'role': 'user', 'content': 'Q:请帮我查下端木天给我发送了哪些邮件，并对这些邮件内容进行总结。'},
 {'role': 'assistant', 'content': 'A:1.请帮我整理端木天发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'},
 {'role': 'user', 'content': 'Q:请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'assistant', 'content': 'A:1.请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'user', 'content': 'Q:请请分析我邮箱里全部已收到的邮件，告诉我最

对其进行任务拆解：

In [134]:
response = openai.ChatCompletion.create(
                model='gpt-4-0613',
                messages=messages
            )

In [135]:
res = response.choices[0].message['content']
res

'A:1.请查找并整理我邮箱里所有的未读邮件列表。2.根据某份邮件列表，筛选出其中和会议邀请有关的邮件。3.对某些给定的邮件，添加一个名称为会议预定的标签。'

这里模型将原始需求拆分为了三个子任务。当然对于我们即将要执行的方案三来说，无论拆分多少个子任务，都是可以依次创建对应部分代码并进行合并的。

&emsp;&emsp;接下来，我们即可采用如下正则表达式提取拆分出来的每个子任务：

In [136]:
# 使用正则表达式查找以1、2、3开始的句子
matches = re.findall(r'\d\.(.*?。)', res)

for match in matches:
    print(match)

请查找并整理我邮箱里所有的未读邮件列表。
根据某份邮件列表，筛选出其中和会议邀请有关的邮件。
对某些给定的邮件，添加一个名称为会议预定的标签。


这里我们将其分别定义：

In [137]:
sub_req1 = '请查找并整理我邮箱里所有的未读邮件列表。'
sub_req2 = '根据某份邮件列表，筛选出其中和会议邀请有关的邮件。'
sub_req3 = '对某些给定的邮件，添加一个名称为会议预定的标签。'

至此，我们就完成了原始任务的拆解部分工作。当然，我们也可以将上述过程封装为一个完整的函数，方便之后进行灵活调用：

In [5]:
def get_decompose_results(req, model='gpt-4-0613'):
    """
    复杂需求拆解函数，能够将用户输入的复杂需求拆解为一系列更容易完成的子任务
    :param req: 必选参数，以字符串形式表示，用于表示用户输入的原始需求；
    :param model: 拆解需求所使用的大模型；
    :return：由子任务所组成的列表；
    """
    
    decompose_results = []
    
    with open('./functions/tested functions/decompose_messages.json', 'r') as f:
        decompose_messages = json.load(f)
        
    decompose_messages.append({"role": "user", "content": req})
    
    response = openai.ChatCompletion.create(
                model=model,
                messages=decompose_messages
            )
    
    res = response.choices[0].message['content']
    
    # 使用正则表达式查找以1、2、3开始的句子
    matches = re.findall(r'\d\.(.*?。)', res)

    for match in matches:
        decompose_results.append(match)
        
    return decompose_results

In [66]:
req

'请查阅我邮箱里面未读邮件，从挑选有关会议邀请的邮件，并将其添加一个名称为会议预定的标签'

In [67]:
get_decompose_results(req)

['请帮我整理我的未读邮件列表。', '根据某份邮件列表，筛选出会议邀请的邮件。', '对某些给定邮件添加会议预定的标签。']

- Stage 2.子任务函数编写与汇总

&emsp;&emsp;接下来，我们即可围绕每个子任务进行函数编写。这里需要注意，正如此前所说，由于某些子任务的需求需要用到此前一个子需求的输出结果，因此目前需求描述其实并不明确，例如'将某份挑选好的邮件添加上会议预定的标签。'这里的某份目前还没被筛选出来，因此单独依靠这个需求，是无法创建一个能够通过function_test测试的函数的，外加当前阶段创建的子任务函数，最终都是要进行最终的整理、汇总以及测试的，因此目前每个子函数的代码正确与否并不会对最终函数有根本性的影响，因此这里我们考虑直接采用code_generate函数进行函数创建，并跳过测试环节。

&emsp;&emsp;接下来使用code_generate函数创建三个子需求的函数，创建过程如下：

In [138]:
func1_name = code_generate(sub_req1, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
def get_unread_emails(userId='me'):
    """
    根据给定的userId，查询Gmail邮箱中所有未读的邮件
    :param userId: 非必要参数，默认为'me'，表示查询调用此函数的用户在Gmail中的未读邮件
    :return 其结果是一个json对象，包含所有未读邮件的列表
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 的 service 客户对象
    service = build('gmail', 'v1', credentials=creds)

    # 列出用户的所有邮件
    results = service.users().messages().list(userId=userId, q='is:unread').execute()
    messages = results.get('messages', [])

    unread_emails = []
    if not messages:
        return json.dumps({"error":"No unread emails found."})
    for message in messages:
        msg = service.users().messages().get(userId=userId, id=message['id']).execute()
        email_data = msg['payload']['headers']
        for d in email_data:
            if d['name'] == 'Subject':
                sub = d['value'

In [139]:
func2_name = code_generate(sub_req2, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
def find_meeting_invitations(query, email_list):
    """
    根据传入的邮件列表，筛选出其中和会议邀请相关的邮件。

    参数:
    query: 要筛选的邮件内容字符串。
    email_list: 要筛选的邮件列表，列表项为字典，其中包含邮件的主要信息。

    返回:
    一个列表，其中每个元素是一个字典，表示与会议活动相关的一封邮件。每个字典包含以下键：
    'From': 发件人的邮箱地址。
    'Date': 邮件的发送日期。
    'Subject': 邮件的主题。
    'Snippet': 邮件的摘要（前100个字符）。

    将查询到的邮件列表以json格式对象返回。

    注意：
    email_list是一个邮件列表，每封邮件的信息都以字典形式储存在列表中；
    字典需要包含邮件的"From"、"Date"、"Subject" 和 "Snippet" 信息；
    若不包含这些信息可能会导致函数出错。
    """

    related_emails = []
    for email in email_list:
        if query.lower() in email['Subject'].lower() or query.lower() in email['Snippet'].lower():
            related_emails.append(email)

    return json.dumps(related_emails)
新函数保存在./functions/untested functions/find_meeting_invitations/find_meeting_invitations_module.py文件中
新函数提示示例保存在./functions/untested functions/find_meeting_

In [140]:
func3_name = code_generate(sub_req3, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
def add_label_to_email(target_mails, label_name):
    """
    给指定的邮件添加一个标签。

    参数:
    target_mails: 需要添加标签的邮件的ID或者列表。
    label_name: 需要添加的标签的名字。

    返回:
    处理完成之后的邮件状态，返回结果本身是json格式对象。
    """

    # 列出指定用户的所有标签
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    # 获取所需标签的名字
    for label in labels:
        if label['name'] == label_name:
            label_id = label['id']
            break
    
    # 如果标签不存在，则创建该标签
    if label_id is None:
        label_obj = {
            'name': label_name
        }
        label = service.users().labels().create(userId='me', body=label_obj).execute()
        label_id = label['id']

    # 为邮件添加标签
    add_label = {
        'addLabelIds': [label_id],
        'removeLabelIds': []
    }
    response = service.users().messages().modify(userId='me', id=target_ma

&emsp;&emsp;三个子函数创建过程并不复杂，接下来考虑将其进行合并。正如此前所说，一个最简单的合并方法就是简单的让大模型对这几个函数的代码进行合并，并且要求代码通畅可运行，同时可以满足复杂用户需求，这里我们采用如下方式将函数源码转化为字符串格式对象：

In [14]:
with open('./functions/untested functions/%s/%s_module.py' % (func1_name, func1_name), encoding='utf-8') as f:
    func1_str = f.read()
    
with open('./functions/untested functions/%s/%s_module.py' % (func2_name, func2_name), encoding='utf-8') as f:
    func2_str = f.read()
    
with open('./functions/untested functions/%s/%s_module.py' % (func3_name, func3_name), encoding='utf-8') as f:
    func3_str = f.read()

然后输入到大模型中，令其读取这三段代码并对其进行合并：

In [146]:
response = openai.ChatCompletion.create(
  model='gpt-3.5-turbo-16k-0613',
  messages=[
      {"role": "system", "content": '以下是函数1的代码：%s' % func1_str},
      {"role": "system", "content": '以下是函数2的代码：%s' % func2_str},
      {"role": "system", "content": '以下是函数3的代码：%s' % func3_str},
      {"role": "user", "content": '请将函数1、函数2和函数3的代码功能整合到一个函数中，要求该函数功能完整，并且可以满足以下需求：%s' % req}
  ]
)

得到结果如下：

In [147]:
s = response.choices[0].message['content']
s

'以下是整合后的函数代码：\n\n```python\ndef process_emails():\n    """\n    从Gmail邮箱中获取未读邮件，并筛选出与会议邀请相关的邮件，然后将其加上名为会议预定的标签。\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 的 service 客户对象\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 获取未读邮件\n    results = service.users().messages().list(userId=\'me\', q=\'is:unread\').execute()\n    messages = results.get(\'messages\', [])\n\n    unread_emails = []\n    if not messages:\n        return json.dumps({"error":"No unread emails found."})\n    for message in messages:\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        email_data = msg[\'payload\'][\'headers\']\n        for d in email_data:\n            if d[\'name\'] == \'Subject\':\n                sub = d[\'value\']\n        unread_emails.append({"id": message[\'id\'], \n                              "subject": sub, \n                              "snippet": msg[\'s

然后使用extract_function_code函数提取代码源码并进行运行：

In [148]:
extract_function_code(s, detail=1, tested=False, g=globals())

def process_emails():
    """
    从Gmail邮箱中获取未读邮件，并筛选出与会议邀请相关的邮件，然后将其加上名为会议预定的标签。
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')

    # 创建 Gmail API 的 service 客户对象
    service = build('gmail', 'v1', credentials=creds)

    # 获取未读邮件
    results = service.users().messages().list(userId='me', q='is:unread').execute()
    messages = results.get('messages', [])

    unread_emails = []
    if not messages:
        return json.dumps({"error":"No unread emails found."})
    for message in messages:
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        email_data = msg['payload']['headers']
        for d in email_data:
            if d['name'] == 'Subject':
                sub = d['value']
        unread_emails.append({"id": message['id'], 
                              "subject": sub, 
                              "snippet": msg['snippet']})

    # 筛选与会议邀请相关的邮件
    query = '会议邀请'
    related_emails = []
    for 

'process_emails'

能够发现，由于该函数的创建过程没有提醒寻找用户需求中变量的环节，因此导致最终创建的函数并不包含任何参数，直接运行即可完成当前需求。这里我们简单运行该函数，测试能否将未读邮件中包含会议邀请的邮件标记为会议预定。在运行函数前，邮箱里包含一封来自端木天的会议邀请邮件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/8753ed817977862b67b97a10d15f5af.png" alt="8753ed817977862b67b97a10d15f5af" style="zoom:33%;" />

接下来运行该函数：

In [149]:
process_emails()

'[{"id": "189c680ef21a1940", "subject": "\\u4f1a\\u8bae\\u9080\\u8bf7", "snippet": "\\u4f1a\\u8bae\\u9080\\u8bf7 \\u9080\\u8bf7\\u60a88\\u670821\\u65e5\\u4e0a\\u5348\\u53c2\\u52a0\\u6280\\u672f\\u90e8\\u7814\\u8ba8\\u4f1a\\u3002"}, {"id": "189bfa28e31ab424", "subject": "\\u4f1a\\u8bae\\u9080\\u8bf7", "snippet": "\\u9080\\u8bf7\\u60a88\\u670821\\u53f7\\u4e0a\\u534810\\u70b9\\u53c2\\u52a0\\u7814\\u53d1\\u8ba1\\u5212\\u8ba8\\u8bba\\u4f1a\\u8bae\\u3002"}]'

运行结束后，该邮件被顺利标记为会议预定：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1d2ad1231ff371f4215fcfceab474c9.png" alt="1d2ad1231ff371f4215fcfceab474c9" style="zoom:33%;" />

同时，我们也可以在会议预定标签下看到该邮件：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/5a42b82cf9b43b30539c989a8b27531.png" style="zoom:33%;" />

至此，得以验证该函数可行性，同时也说明简化版的方案三本身的可行性。

&emsp;&emsp;不过，需要注意的是，实际操作过程中，这种合并方式的准确率其实和子函数的代码质量有很大的关系，需要注意的是，此时由于大模型由于缺少了LtM提示流程，因此对原始编程需求理解的并不深，只能根据子函数的代码来推断原始需求的实现过程，因此一旦子函数本身质量不高，最终合并得到的函数的准确率也会比较低。而同时我们也知道，子函数的创建本身就是没有审核的，因此子函数的代码质量也极有可能是层次不齐的，因此总的来看，这种合并方式仍然存在一些不太可控的风险。

#### 3.4 复杂需求的自动自然语言编程流程

&emsp;&emsp;而对于大模型工程师来说，核心职能之一就是应用好大模型的一些不稳定的特性，来围绕某些问题提供非常稳定的解决方案。因此接下来我们尝试按照严格的code_generate+function_test流程来进行子函数的合并，并进行准确性更好的原始需求函数编写流程。当然，在这个流程中，子函数合并其实指的是将子函数视作code_generate第三阶段提示的system message用于辅助函数编写，并不是简单的审查与合并。需要注意的是，我们在创建code_generate函数时，其实是将每个阶段的system messages写死了的，我们无法通过外部参数来调整各阶段的system message。这里在code_generate函数中的system message部分代码如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/58affbb4f67d821d8fba6e5eb0f71bf.png" alt="58affbb4f67d821d8fba6e5eb0f71bf" style="zoom:33%;" />

因此，我们需要修改此前定义的code_generate函数，为其增加一个system_messages参数。该参数默认取值为None，表示导入本地保存的system_messages.json对象，否则输入外部messages作为system_messages:

In [45]:
def code_generate(req, few_shot='all', model='gpt-4-0613', g=globals(), detail=0, system_messages=None):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4-0613；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :param system_messages: 可选参数，默认取值为None，此时读取本地system_messages.json作为系统信息，也可以自定义；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    if system_messages == None:
        with open('./functions/tested functions/system_messages.json', 'r') as f:
            system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages_CD
                )
    new_req_pi = response.choices[0].message['content']
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = openai.ChatCompletion.create(
                      model=model,
                      messages=few_shot_messages_CM
                    )
    new_req_description = response.choices[0].message['content']
    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = openai.ChatCompletion.create(
                  model=model,
                  messages=few_shot_messages
                )
    new_req_function = response.choices[0].message['content']
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

而当我们要执行code_generate+function_test流程来进行函数合并时，需要在原始的system_messages中的第三阶段系统提示添加任务拆解结果和子函数源码：

In [15]:
with open('./functions/tested functions/system_messages.json', 'r') as f:
    system_messages = json.load(f)

In [16]:
system_messages

{'system_message_CD': [{'role': 'system',
   'content': '为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。'}],
 'system_message_CM': [{'role': 'system',
   'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'}],
 'system_message': [{'role': 'system',
   'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'}]}

In [17]:
system_messages['system_message']

[{'role': 'system',
  'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'}]

In [20]:
decompose_description = ('对于当前编程需求，可以拆解为若干个子需求，也就是：%s。这些子需求的实现方式可以参考：' + func1_str + func2_str + func3_str) % res[2:]

In [21]:
decompose_description

'对于当前编程需求，可以拆解为若干个子需求，也就是：1.请查找并整理我邮箱里所有的未读邮件列表。2.根据某份邮件列表，筛选出其中和会议邀请有关的邮件。3.对某些给定的邮件，添加一个名称为会议预定的标签。。这些子需求的实现方式可以参考：def get_unread_emails(userId=\'me\'):\n    """\n    根据给定的userId，查询Gmail邮箱中所有未读的邮件\n    :param userId: 非必要参数，默认为\'me\'，表示查询调用此函数的用户在Gmail中的未读邮件\n    :return 其结果是一个json对象，包含所有未读邮件的列表\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 的 service 客户对象\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的所有邮件\n    results = service.users().messages().list(userId=userId, q=\'is:unread\').execute()\n    messages = results.get(\'messages\', [])\n\n    unread_emails = []\n    if not messages:\n        return json.dumps({"error":"No unread emails found."})\n    for message in messages:\n        msg = service.users().messages().get(userId=userId, id=message[\'id\']).execute()\n        email_data = msg[\'payload\'][\'headers\']\n        for d in email_data:\n            if d[\'name\'] == \'Subject\':

In [22]:
system_messages['system_message'].append({'role': 'system', 'content': decompose_description})

In [23]:
system_messages

{'system_message_CD': [{'role': 'system',
   'content': '为了更好编写满足用户需求的python函数，我们需要先识别用户需求中的变量，以作为python函数的参数。需要注意的是，当前编写的函数中涉及到的邮件收发查阅等功能，都是通过调用Gmail API来完成。'}],
 'system_message_CM': [{'role': 'system',
   'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'}],
 'system_message': [{'role': 'system',
   'content': '我现在已完成Gmail API授权，授权文件为本地文件token.json。函数参数必须是字符串类型对象，函数返回结果必须是json表示的字符串对象。'},
  {'role': 'system',
   'content': '对于当前编程需求，可以拆解为若干个子需求，也就是：1.请查找并整理我邮箱里所有的未读邮件列表。2.根据某份邮件列表，筛选出其中和会议邀请有关的邮件。3.对某些给定的邮件，添加一个名称为会议预定的标签。。这些子需求的实现方式可以参考：def get_unread_emails(userId=\'me\'):\n    """\n    根据给定的userId，查询Gmail邮箱中所有未读的邮件\n    :param userId: 非必要参数，默认为\'me\'，表示查询调用此函数的用户在Gmail中的未读邮件\n    :return 其结果是一个json对象，包含所有未读邮件的列表\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n\n    # 创建 Gmail API 的 service 客户对象\n    service = build(\'gmail\', \'v1\', credentials=creds)\n\n    # 列出用户的所有邮件\n    results = service.us

然后以此作为system_message带入到code_generate中进行代码创建：

In [32]:
code_generate(req, few_shot='all', model='gpt-4-0613', g=globals(), detail=0, system_messages=system_messages)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:label_meeting_emails
新函数保存在./functions/untested functions/label_meeting_emails/label_meeting_emails_module.py文件中
新函数提示示例保存在./functions/untested functions/label_meeting_emails/label_meeting_emails_prompt.json文件中
done


'label_meeting_emails'

In [33]:
label_meeting_emails?

Signature: label_meeting_emails(query, label, userId='me')
Docstring:
查找未读邮件中有关会议邀请的邮件并添加“会议预定”标签。

参数:
query: 要查找的邮件类型。
label: 要添加的标签名称。
userId: 查询并标记邮件的用户ID，默认值是'me'。

返回:
已经添加标签的未读邮件的数量，结果会返回为json对象。
File:      Dynamically generated function. No source code available.
Type:      function

接下来验证函数性能：

In [34]:
messages = [{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"},
            {"role": "system", "content": "我的邮箱地址是:ksken166@gmail.com"},
            {"role": "user", "content": req}]

In [35]:
run_conversation(messages, functions_list=[label_meeting_emails], model="gpt-4-0613")

'已完成，找到1封有关会议邀请的未读邮件，并已添加"会议预定"的标签。'

通过查阅邮箱，我们发现借助该函数，大模型已将一封新的会议邀请邮件添加上了会议预定标签：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/2f870254ad099f05b49199bf7f982e5.png" alt="2f870254ad099f05b49199bf7f982e5" style="zoom:33%;" />

至此，也验证了创建流程和最终所创建函数本身的正确性。

&emsp;&emsp;这里需要注意的是，我们之所以考虑采用code_generate函数进行对应代码创建，核心原因在于该函数会创建一份完整的prompt提示示例，便于搭配function_test对其进行审查。接下来，我们就尝试围绕修改function_test的测试流程，是的其能够对拆分多个子需求之后的函数创建过程进行审查。

&emsp;&emsp;首先需要修改的就是prompt_modified函数，为了更好的协调function_test和code_generate共同进行代码创建和审查，我们需要将这些函数的相关参数进行统一，这里对于prompt_modified函数的修改主要是添加一个system_messages参数，用于表示此时是围绕简单需求进行提示审查还是围绕拆解需求并进行分段代码创建的流程进行提示审查，和code_generate函数类似，默认情况都是取值为None，表示围绕简单需求进行提示审查。而当system_messages取值不为None时，则会提供一个名为《复杂需求推理链修改》的文档作为提示审查函数外部挂载文档，相比《推理链修改》文档，该文档只修改了一个部分，即将原先的作为正确示例的prompt.json文档修改为此前我们成功创建的label_meeting_emails函数对应的提示示例，该函数的提示示例（配合复杂需求拆解的system_message）能够更好的表示复杂需求拆解和代码合并流程：

In [12]:
 with open('./functions/untested functions/label_meeting_emails/label_meeting_emails_prompt.json' , 'r') as f:
    data = json.load(f)

In [13]:
data

{'stage1_CD': [{'role': 'user',
   'content': '请查阅我邮箱里面未读邮件，从挑选有关会议邀请的邮件，并将其添加一个名称为会议预定的标签'},
  {'role': 'assistant',
   'content': '当前需求中可以作为函数参数的是：1.查询条件；2.添加的标签名；3.查询谁的邮箱'}],
 'stage1_CM': [{'role': 'user',
   'content': '请查阅我邮箱里面未读邮件，从挑选有关会议邀请的邮件，并将其添加一个名称为会议预定的标签当前需求中可以作为函数参数的是：1.查询条件；2.添加的标签名；3.查询谁的邮箱'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，查询我收到的所有未读邮件，并在与会议邀请相关的邮件上添加一种名为“会议预定”的标签，函数要求如下：                  1.函数参数query、tag和user_id，三个参数都是字符串类型，其中query表示要查询的条件，tag表示要添加的标签名，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；                  2.函数返回结果是添加标签后的邮件列表，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，查询我收到的所有未读邮件，并在与会议邀请相关的邮件上添加一种名为“会议预定”的标签，函数要求如下：                  1.函数参数query、tag和user_id，三个参数都是字符串类型，其中query表示要查询的条件，tag表示要添加的标签名，user_id表示要检索邮件的用户的ID。默认值是'me'，表示当前授权的用户；                  2.函数返回结果是添加标签后的邮件列表，返回结果本身必须是一个jso

我们将《推理链修改》文档中的正确提示流程换作上述data，即可完成《复杂需求推理链修改》文档编写

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/a971634400dc863740b5939e89e98d6.png" alt="a971634400dc863740b5939e89e98d6" style="zoom:33%;" />

具体prompt_modified函数修改如下：

In [46]:
def prompt_modified(function_name, system_content='推理链修改.md', model="gpt-4-0613", g=globals(), system_messages=None):
    """
    智能邮件项目的外部函数审查函数，用于审查外部函数创建流程提示是否正确以及最终创建的代码是否正确
    :param function_name: 必要参数，字符串类型，表示审查对象名称；
    :param system_content: 可选参数，默认取值为字符串推理链修改.md，表示此时审查函数外部挂载文档名称，需要是markdwon格式文档；
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4-0613；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param system_messages: 可选参数，默认取值为None，此时读取本地system_messages.json作为系统信息，也可以自定义；
    :return：审查结束后新创建的函数名称
    """
    print("正在执行审查函数，审查对象：%s" % function_name)
    
    if system_messages != None:
        system_content='复杂需求推理链修改.md'
        
    with open(system_content, 'r', encoding='utf-8') as f:
        md_content = f.read()  
        
    # 读取原函数全部提示内容
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    
    # 将其保存为字符串
    msg_str = json.dumps(msg)
    
    # 进行审查
    response = openai.ChatCompletion.create(
                    model=model,
                    messages=[
                    {"role": "system", "content": md_content},
                    {"role": "user", "content": '以下是一个错误的智能邮件项目的推理链，请你按照要求对其进行修改：%s' % msg_str}
                    ]
                )
    
    modified_result = response.choices[0].message['content']
    
    def extract_json(s):
        pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
        match = re.search(pattern, s, re.DOTALL)
        if match:
            return match.group(1)
        else:
            return s
    
    modified_json = extract_json(modified_result)
    
    # 提取函数源码
    code = json.loads(modified_json)['stage2'][1]['content']
    
    # 提取函数名
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    print("审查结束，新的函数名称为：%s。\n正在运行该函数定义过程，并保存函数源码与prompt" % function_name)
    
    exec(code, g)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # 写入函数
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
        f.write(code)
        
    # 写入提示
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(json.loads(modified_json), f)
    
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print("%s函数已在当前操作空间定义，可以进行效果测试" % function_name)
    
    return function_name

能够看出，此时system_messages对于该函数来说，更像是一个标识符，在该函数实际执行审查时，并不会真的带入新的系统消息进行审查。

&emsp;&emsp;据此，我们就可以进一步围绕function_test函数进行修改。该函数的修改过程并不复杂，核心是需要将审查过程所涉及的一系列的函数（包括自己）添加一个system_messages的参数，同样，默认情况下取值为None，其他时候该参数的取值将输入到code_genetate和prompt_modified函数中：

In [83]:
def function_test(function_name, req, few_shot, model="gpt-4-0613", g=globals(), system_messages=None):

    def test_messages(ueser_content):
        messages = [{"role": "system", "content": "端木天的邮箱地址是:2323365771@qq.com"},
                    {"role": "system", "content": "我的邮箱地址是:ksken166@gmail.com"},
                    {"role": "user", "content": ueser_content}]
        return messages
            
    messages = test_messages(req)
    
    new_function = globals()[function_name]
    functions_list = [new_function]
    
    print("根据既定用户需求req进行%s函数功能测试，请确保当该函数已经在当前操作空间定义..." % function_name)
    
    # 有可能在run_conversation环节报错
    # 若没报错，则运行：
    try:
        final_response = run_conversation(messages=messages, functions_list=functions_list, model=model)
        print("当前函数运行结果：'%s'" % final_response)
        
        feedback = input("函数功能是否满足要求 (yes/no)? ")
        if feedback.lower() == 'yes':
            print("函数功能通过测试，正在将函数写入tested文件夹")
            remove_to_tested(function_name)
            print('done')
        else:
            next_step = input("函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？")
            if next_step == '1':
                print("准备再次测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
            else:
                solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
                \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
                if solution == '1':
                    # 再次运行函数创建过程
                    print("好的，正在尝试再次创建函数，请稍等...")
                    few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                    if few_shot_str == '1':
                        Gmail_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        Gmail_auto_func(req=req, few_shot='all', model=model, g=g)
                elif solution == '2':
                    # 执行审查函数
                    print("好的，执行审查函数，请稍等...")
                    function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g, system_messages=system_messages)
                    # 接下来带入进行测试
                    print("新函数已创建，接下来带入进行测试...")
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
                elif solution == '3':
                    new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                    few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                    if few_shot_str == '1':
                        Gmail_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        Gmail_auto_func(req=req, few_shot='all', model=model, g=g)
                elif solution == '4':
                    print("好的，预祝debug顺利~")
        
    # run_conversation报错时则运行：
    except Exception as e:
        next_step = input("run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？")
        if next_step == '1':
            function_test(function_name=function_name, req=new_req, few_shot=few_shot, g=g, system_messages=system_messages)
        else:
            solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
            \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
            if solution == '1':
                # 再次运行函数创建过程
                print("好的，正在尝试再次创建函数，请稍等...")
                few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                if few_shot_str == '1':
                    Gmail_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    Gmail_auto_func(req=req, few_shot='all', model=model, g=g)
            elif solution == '2':
                # 执行审查函数
                print("好的，执行审查函数，请稍等...")
                max_attempts = 3
                attempts = 0

                while attempts < max_attempts:
                    try:
                        function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g, system_messages=system_messages)
                        break  # 如果代码成功执行，跳出循环
                    except Exception as e:
                        attempts += 1  # 增加尝试次数
                        print("发生错误：", e)
                        if attempts == max_attempts:
                            print("已达到最大尝试次数，程序终止。")
                            raise  # 重新引发最后一个异常
                        else:
                            print("正在重新运行审查程序...")
                # 接下来带入进行测试
                print("新函数已创建，接下来带入进行测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
            elif solution == '3':
                new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                if few_shot_str == '1':
                    Gmail_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    Gmail_auto_func(req=req, few_shot='all', model=model, g=g)
            elif solution == '4':
                print("好的，预祝debug顺利~")

&emsp;&emsp;至此，我们就完成了code_generate函数和function_test函数修改。当然，为了让整个自动编程流程能够更加自动，接下来我们进一步对Gmail_auto_func函数进行修改。此时该函数不仅承担同时运行code_generate函数和function_test函数的功能，更承担着对原始任务进行拆解的功能，具体函数修改内容如下：

In [82]:
def Gmail_auto_func(req, few_shot='all', model='gpt-4-0613', g=globals(), detail=0):
    # 默认情况下system_messages = None
    system_messages = None
    
    # 尝试进行任务拆解
    try:
        decompose_results = get_decompose_results(req=req, model=model)
    except Exception as e:
        print(e)
        print('暂停1分钟后继续调用模型')
        time.sleep(60)
        decompose_results = get_decompose_results(req=req, model=model)
        
    # 如果只拆解得到多个任务，则创建新的基于任务拆解得到的system_message
    if len(decompose_results) != 1:
        print('原始需求将拆分为多个子需求并进行分段代码创建与合并')
        # 读取原始system_message
        with open('./functions/tested functions/system_messages.json', 'r') as f:
            system_messages = json.load(f)
            
        # 用于存储全部需求的函数代码
        sub_func_all = ''
        # 用于存储全部需求
        sub_req_all = ''
        # 计数器
        i = 0
        
        for sub_req in decompose_results:
            i += 1
            # 每个需求依次创建子函数
            print('第%s个子需求为：%s' % (i, sub_req))
            print('正在创建对应子函数')
            try:
                sub_func_name = code_generate(sub_req, few_shot=few_shot, g=g, detail=detail, model=model)
            except Exception as e:
                print(e)
                print('暂停1分钟后继续调用模型')
                time.sleep(60)
                sub_func_name = code_generate(sub_req, few_shot=few_shot, g=g, detail=detail, model=model)
                
            # 读取子函数源码
            with open('./functions/untested functions/%s/%s_module.py' % (sub_func_name, sub_func_name), encoding='utf-8') as f:
                sub_func_str = f.read()
            # 对子函数源码进行拼接
            sub_func_all += sub_func_str
            
            # 按顺序拼接子需求
            sub_req_all += str(i)+('.')
            sub_req_all += sub_req            
        
        print('子需求对应的子函数全部创建完毕，接下来进入到原始需求函数创建过程...')
        # 添加一个system_message
        decompose_description = '对于当前编程需求，可以拆解为若干个子需求，也就是：%s。这些子需求的实现方式可以参考如下代码：%s' % (sub_req_all, sub_func_all)
        system_messages['system_message'].append({'role': 'system', 'content': decompose_description})
    
    # 进行代码创建和代码审查
    try:
        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g, detail=detail, system_messages=system_messages)
    except Exception as e:
        print(e)
        print('暂停1分钟后继续调用模型')
        time.sleep(60)
        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g, detail=detail, system_messages=system_messages)
    function_test(function_name=function_name, req=req, few_shot=few_shot, model=model, g=g, system_messages=system_messages)

这里需要注意，由于整体代码执行过程中存在反复多次调用gpt模型的情况，因此极有可能在运行过程中遇到调用次数或者token上限，会引发如下错误。因此在代码中添加了遇到如下类型报错则暂停一分钟的代码。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/fc07fa61e89e76249d603adadccf312.png" alt="fc07fa61e89e76249d603adadccf312" style="zoom:33%;" />

此外，为了更好的展示当前代码创建进度，和code_generate函数类似，我们在该函数内部设置了多个关键环节打印流程。当创建完成之后，接下来即可对该函数进行功能测试。这里我们重新给出一个新的需求，测试函数创建功能：

In [84]:
req = '在我的邮箱里，从未读邮件中挑选和预算相关的邮件，并为这些邮件添加“预算”的标签'

In [5]:
Gmail_auto_func(req=req)

原始需求将拆分为多个子需求并进行分段代码创建与合并
第1个子需求为：请帮我查找邮箱内的所有未读邮件。
正在创建对应子函数
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_unread_emails
新函数保存在./functions/untested functions/get_unread_emails/get_unread_emails_module.py文件中
新函数提示示例保存在./functions/untested functions/get_unread_emails/get_unread_emails_prompt.json文件中
done
第2个子需求为：从某份邮件列表中挑选出有关预算相关的邮件。
正在创建对应子函数
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:search_email_by_keyword
新函数保存在./functions/untested functions/search_email_by_keyword/search_email_by_keyword_module.py文件中
新函数提示示例保存在./functions/untested functions/search_email_by_keyword/search_email_by_keyword_prompt.json文件中
done
第3个子需求为：对某些特定邮件批量添加名称为预算的标签。
正在创建对应子函数
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行

测试效果如下

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/2169d10fbc76fbf2bd0da625cf90d3c.png" alt="2169d10fbc76fbf2bd0da625cf90d3c" style="zoom:33%;" />

<center>

<img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/ad08239a9b27796ce5b7420bfe68cbe.png" alt="ad08239a9b27796ce5b7420bfe68cbe" style="zoom:33%;" />

- 复杂需求自然语言编程流程总结

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/%E5%A4%8D%E6%9D%82%E7%BC%96%E7%A8%8B%E9%9C%80%E6%B1%82%E5%AE%9E%E7%8E%B0%E6%B5%81%E7%A8%8B.png" alt="复杂编程需求实现流程" style="zoom:33%;" />